In [15]:
file_name = '0111295000.jpeg'

import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from PIL import Image
import cv2
import os
import math
import sys
import numpy as np
import pytesseract
import io 
import csv
import pandas as pd
import shutil, os
import warnings
warnings.filterwarnings('ignore')

shutil.copytree('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input/sample', 
    os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input',
                             os.path.splitext(file_name)[0]))
img = mpimg.imread(os.path.join
                   ('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input/input',
                    file_name))
width = img.shape[1]
height = img.shape[0]
wi = width * 0.95
he = height * 0.1
w = int(wi)
h = int(he)
y = 350
x1 = w * 0.45
x = int(x1)
crop = img[y:y+h, x:x+w]
im = Image.fromarray(crop)
b = os.path.splitext(file_name)[0]+'_crop.jpeg'
im.save(os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input',
                       os.path.splitext(file_name)[0],'crop',b))

def extract_cell_images_from_table(image):
    BLUR_KERNEL_SIZE = (17, 17)
    STD_DEV_X_DIRECTION = 0
    STD_DEV_Y_DIRECTION = 0
    blurred = cv2.GaussianBlur(image, BLUR_KERNEL_SIZE, STD_DEV_X_DIRECTION, STD_DEV_Y_DIRECTION)
    MAX_COLOR_VAL = 255
    BLOCK_SIZE = 15
    SUBTRACT_FROM_MEAN = -2
    
    img_bin = cv2.adaptiveThreshold(
        ~blurred,
        MAX_COLOR_VAL,
        cv2.ADAPTIVE_THRESH_MEAN_C,
        cv2.THRESH_BINARY,
        BLOCK_SIZE,
        SUBTRACT_FROM_MEAN,
    )
    vertical = horizontal = img_bin.copy()
    SCALE = 5
    image_width, image_height = horizontal.shape
    horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (int(image_width / SCALE), 1))
    horizontally_opened = cv2.morphologyEx(img_bin, cv2.MORPH_OPEN, horizontal_kernel)
    vertical_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, int(image_height / SCALE)))
    vertically_opened = cv2.morphologyEx(img_bin, cv2.MORPH_OPEN, vertical_kernel)
    
    horizontally_dilated = cv2.dilate(horizontally_opened, cv2.getStructuringElement(cv2.MORPH_RECT, (40, 1)))
    vertically_dilated = cv2.dilate(vertically_opened, cv2.getStructuringElement(cv2.MORPH_RECT, (1, 60)))
    
    mask = horizontally_dilated + vertically_dilated
    contours, heirarchy = cv2.findContours(
        mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE,
    )
    
    perimeter_lengths = [cv2.arcLength(c, True) for c in contours]
    epsilons = [0.05 * p for p in perimeter_lengths]
    approx_polys = [cv2.approxPolyDP(c, e, True) for c, e in zip(contours, epsilons)]
    
    # Filter out contours that aren't rectangular. Those that aren't rectangular
    # are probably noise.
    approx_rects = [p for p in approx_polys if len(p) == 4]
    bounding_rects = [cv2.boundingRect(a) for a in approx_polys]
    
    # Filter out rectangles that are too narrow or too short.
    MIN_RECT_WIDTH = 40
    MIN_RECT_HEIGHT = 10
    bounding_rects = [
        r for r in bounding_rects if MIN_RECT_WIDTH < r[2] and MIN_RECT_HEIGHT < r[3]
    ]
    
    # The largest bounding rectangle is assumed to be the entire table.
    # Remove it from the list. We don't want to accidentally try to OCR
    # the entire table.
    largest_rect = max(bounding_rects, key=lambda r: r[2] * r[3])
    bounding_rects = [b for b in bounding_rects if b is not largest_rect]
    
    cells = [c for c in bounding_rects]
    def cell_in_same_row(c1, c2):
        c1_center = c1[1] + c1[3] - c1[3] / 2
        c2_bottom = c2[1] + c2[3]
        c2_top = c2[1]
        return c2_top < c1_center < c2_bottom
    
    orig_cells = [c for c in cells]
    rows = []
    while cells:
        first = cells[0]
        rest = cells[1:]
        cells_in_same_row = sorted(
            [
                c for c in rest
                if cell_in_same_row(c, first)
            ],
            key=lambda c: c[0]
        )
    
        row_cells = sorted([first] + cells_in_same_row, key=lambda c: c[0])
        rows.append(row_cells)
        cells = [
            c for c in rest
            if not cell_in_same_row(c, first)
        ]
    
    # Sort rows by average height of their center.
    def avg_height_of_center(row):
        centers = [y + h - h / 2 for x, y, w, h in row]
        return sum(centers) / len(centers)
    
    rows.sort(key=avg_height_of_center)
    cell_images_rows = []
    for row in rows:
        cell_images_row = []
        for x, y, w, h in row:
            cell_images_row.append(image[y:y+h, x:x+w])
        cell_images_rows.append(cell_images_row)
    return cell_images_rows

def main(f):
    results = []
    directory, filename = os.path.split(f)
    table = cv2.imread(f, cv2.IMREAD_GRAYSCALE)
    rows = extract_cell_images_from_table(table)
    #cell_img_dir = os.path.join(directory, "cells")
    #os.makedirs(cell_img_dir, exist_ok=True)
    out_path = os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input',
                            os.path.splitext(file_name)[0],'cells')
    paths = []
    for i, row in enumerate(rows):
        for j, cell in enumerate(row):
            cell_filename = "{:03d}-{:03d}.png".format(i, j)
            path = os.path.join(out_path, cell_filename)
            cv2.imwrite(path, cell)
            paths.append(path)
    return paths

f = os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input',
                       os.path.splitext(file_name)[0],'crop',b)
main(f)

def main(image_file, tess_args):
    """
    OCR the image and output the text to a file with an extension that is ready
    to be used in Tesseract training (.gt.txt).
    Tries to crop the image so that only the relevant text gets passed to Tesseract.
    Returns the name of the text file that contains the text.
    """
    #file_path = '/home/vimal/Documents/table_detect_samples/structured images/input/011364700/cells'
    #image_file = os.listdir(file_path)
    
    for f in image_file:
        #print(f)
        directory, filename = os.path.split(f)
        filename_sans_ext, ext = os.path.splitext(filename)
        image = cv2.imread(os.path.join(file_path,f), cv2.IMREAD_GRAYSCALE)
        cropped = crop_to_text(image)
        #ocr_data_dir = os.path.join(directory, "ocr_data")
        #os.makedirs(ocr_data_dir, exist_ok=True)
        #out_imagepath = os.path.join(ocr_data_dir, filename)
        out_path = os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input',
                            os.path.splitext(file_name)[0],'text')
        out_txtpath = os.path.join(out_path, "{}.txt".format(filename_sans_ext))
        #cv2.imwrite(out_imagepath, cropped)
        if not tess_args:
            d = os.path.dirname(sys.modules["table_ocr"].__file__)
            tessdata_dir = os.path.join(d, "tessdata")
            tess_args = ["--psm", "7", "-l", "table-ocr", "--tessdata-dir", tessdata_dir]
        txt = ocr_image(cropped, " ".join(tess_args))
        with open(out_txtpath, "w") as txt_file:
            txt_file.write(txt)
        #return out_txtpath
def crop_to_text(image):
    MAX_COLOR_VAL = 255
    BLOCK_SIZE = 15
    SUBTRACT_FROM_MEAN = -2

    img_bin = cv2.adaptiveThreshold(
        ~image,
        MAX_COLOR_VAL,
        cv2.ADAPTIVE_THRESH_MEAN_C,
        cv2.THRESH_BINARY,
        BLOCK_SIZE,
        SUBTRACT_FROM_MEAN,
    )

    img_h, img_w = image.shape
    horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (int(img_w * 0.5), 1))
    vertical_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, int(img_h * 0.7)))
    horizontal_lines = cv2.morphologyEx(img_bin, cv2.MORPH_OPEN, horizontal_kernel)
    vertical_lines = cv2.morphologyEx(img_bin, cv2.MORPH_OPEN, vertical_kernel)
    both = horizontal_lines + vertical_lines
    cleaned = img_bin - both

    # Get rid of little noise.
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    opened = cv2.morphologyEx(cleaned, cv2.MORPH_OPEN, kernel)
    opened = cv2.dilate(opened, kernel)

    contours, hierarchy = cv2.findContours(opened, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    bounding_rects = [cv2.boundingRect(c) for c in contours]
    NUM_PX_COMMA = 6
    MIN_CHAR_AREA = 5 * 9
    char_sized_bounding_rects = [(x, y, w, h) for x, y, w, h in bounding_rects if w * h > MIN_CHAR_AREA]
    if char_sized_bounding_rects:
        minx, miny, maxx, maxy = math.inf, math.inf, 0, 0
        for x, y, w, h in char_sized_bounding_rects:
            minx = min(minx, x)
            miny = min(miny, y)
            maxx = max(maxx, x + w)
            maxy = max(maxy, y + h)
        x, y, w, h = minx, miny, maxx - minx, maxy - miny
        cropped = image[y:min(img_h, y+h+NUM_PX_COMMA), x:min(img_w, x+w)]
    else:
        # If we morphed out all of the text, assume an empty image.
        cropped = MAX_COLOR_VAL * np.ones(shape=(20, 100), dtype=np.uint8)
    bordered = cv2.copyMakeBorder(cropped, 5, 5, 5, 5, cv2.BORDER_CONSTANT, None, 255)
    return bordered
def ocr_image(image, config):
    return pytesseract.image_to_string(
        image,
        lang='eng', config='--psm 6'
    )

file_path = os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input',
                            os.path.splitext(file_name)[0],'cells')
image_file = os.listdir(file_path)
tess_args = os.listdir(file_path)

main(image_file, tess_args)

def text_files_to_csv(files):
    """Files must be sorted lexicographically
    Filenames must be <row>-<colum>.txt.
    000-000.txt
    000-001.txt
    001-000.txt
    etc...
    """
    rows = []
    for f in files:
        directory, filename = os.path.split(f)
        with open(os.path.join(file_path,f)) as of:
            txt = of.read().strip()
        row, column = map(int, filename.split(".")[0].split("-"))
        if row == len(rows):
            rows.append([])
        rows[row].append(txt)

    csv_file = io.StringIO()
    writer = csv.writer(csv_file)
    writer.writerows(rows)
    return csv_file.getvalue()

def main(files):
    return text_files_to_csv(files)


file_path = os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input',
                            os.path.splitext(file_name)[0],'text')
file = os.listdir(file_path)
files = sorted(file)

a = main(files)
#print(a)

c = os.path.splitext(file_name)[0]+'_text.txt'
#cs = os.path.splitext(file_name)[0]+'_xl.csv'

text_file = open(os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input',
                            os.path.splitext(file_name)[0],'txt',c), "wt")
n = text_file.write(a)
text_file.close()

df = pd.read_csv(os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input',
                            os.path.splitext(file_name)[0],'txt',c),
    header=None,delimiter=',',names=list(range(10)))
df1 = df[df[0].str.contains('TOTAL WAGES PAID') | df[0].str.contains('total wages paid')
                             | df[0].str.contains('Total wages paid')]
df1.dropna(axis=1)

FileNotFoundError: [Errno 2] No such file or directory: '/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input/input/0105256000.jpeg'

In [1]:

def total_wages_paid(file_name):
    import matplotlib.image as mpimg
    import matplotlib.pyplot as plt
    from PIL import Image
    import cv2
    import os
    import math
    import sys
    import numpy as np
    import pytesseract
    import io 
    import csv
    import pandas as pd
    import shutil, os
    import warnings
    warnings.filterwarnings('ignore')

    shutil.copytree('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input/sample', 
        os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input',
                                 os.path.splitext(file_name)[0]))
    img = mpimg.imread(os.path.join
                       ('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input/input',
                        file_name))
    width = img.shape[1]
    height = img.shape[0]
    wi = width * 0.95
    he = height * 0.1
    w = int(wi)
    h = int(he)
    y = 350
    x1 = w * 0.45
    x = int(x1)
    crop = img[y:y+h, x:x+w]
    im = Image.fromarray(crop)
    b = os.path.splitext(file_name)[0]+'_crop.jpeg'
    im.save(os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input',
                           os.path.splitext(file_name)[0],'crop',b))

    def extract_cell_images_from_table(image):
        BLUR_KERNEL_SIZE = (17, 17)
        STD_DEV_X_DIRECTION = 0
        STD_DEV_Y_DIRECTION = 0
        blurred = cv2.GaussianBlur(image, BLUR_KERNEL_SIZE, STD_DEV_X_DIRECTION, STD_DEV_Y_DIRECTION)
        MAX_COLOR_VAL = 255
        BLOCK_SIZE = 15
        SUBTRACT_FROM_MEAN = -2

        img_bin = cv2.adaptiveThreshold(
            ~blurred,
            MAX_COLOR_VAL,
            cv2.ADAPTIVE_THRESH_MEAN_C,
            cv2.THRESH_BINARY,
            BLOCK_SIZE,
            SUBTRACT_FROM_MEAN,
        )
        vertical = horizontal = img_bin.copy()
        SCALE = 5
        image_width, image_height = horizontal.shape
        horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (int(image_width / SCALE), 1))
        horizontally_opened = cv2.morphologyEx(img_bin, cv2.MORPH_OPEN, horizontal_kernel)
        vertical_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, int(image_height / SCALE)))
        vertically_opened = cv2.morphologyEx(img_bin, cv2.MORPH_OPEN, vertical_kernel)

        horizontally_dilated = cv2.dilate(horizontally_opened, cv2.getStructuringElement(cv2.MORPH_RECT, (40, 1)))
        vertically_dilated = cv2.dilate(vertically_opened, cv2.getStructuringElement(cv2.MORPH_RECT, (1, 60)))

        mask = horizontally_dilated + vertically_dilated
        contours, heirarchy = cv2.findContours(
            mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE,
        )

        perimeter_lengths = [cv2.arcLength(c, True) for c in contours]
        epsilons = [0.05 * p for p in perimeter_lengths]
        approx_polys = [cv2.approxPolyDP(c, e, True) for c, e in zip(contours, epsilons)]

        # Filter out contours that aren't rectangular. Those that aren't rectangular
        # are probably noise.
        approx_rects = [p for p in approx_polys if len(p) == 4]
        bounding_rects = [cv2.boundingRect(a) for a in approx_polys]

        # Filter out rectangles that are too narrow or too short.
        MIN_RECT_WIDTH = 40
        MIN_RECT_HEIGHT = 10
        bounding_rects = [
            r for r in bounding_rects if MIN_RECT_WIDTH < r[2] and MIN_RECT_HEIGHT < r[3]
        ]

        # The largest bounding rectangle is assumed to be the entire table.
        # Remove it from the list. We don't want to accidentally try to OCR
        # the entire table.
        largest_rect = max(bounding_rects, key=lambda r: r[2] * r[3])
        bounding_rects = [b for b in bounding_rects if b is not largest_rect]

        cells = [c for c in bounding_rects]
        def cell_in_same_row(c1, c2):
            c1_center = c1[1] + c1[3] - c1[3] / 2
            c2_bottom = c2[1] + c2[3]
            c2_top = c2[1]
            return c2_top < c1_center < c2_bottom

        orig_cells = [c for c in cells]
        rows = []
        while cells:
            first = cells[0]
            rest = cells[1:]
            cells_in_same_row = sorted(
                [
                    c for c in rest
                    if cell_in_same_row(c, first)
                ],
                key=lambda c: c[0]
            )

            row_cells = sorted([first] + cells_in_same_row, key=lambda c: c[0])
            rows.append(row_cells)
            cells = [
                c for c in rest
                if not cell_in_same_row(c, first)
            ]

        # Sort rows by average height of their center.
        def avg_height_of_center(row):
            centers = [y + h - h / 2 for x, y, w, h in row]
            return sum(centers) / len(centers)

        rows.sort(key=avg_height_of_center)
        cell_images_rows = []
        for row in rows:
            cell_images_row = []
            for x, y, w, h in row:
                cell_images_row.append(image[y:y+h, x:x+w])
            cell_images_rows.append(cell_images_row)
        return cell_images_rows

    def main(f):
        results = []
        directory, filename = os.path.split(f)
        table = cv2.imread(f, cv2.IMREAD_GRAYSCALE)
        rows = extract_cell_images_from_table(table)
        #cell_img_dir = os.path.join(directory, "cells")
        #os.makedirs(cell_img_dir, exist_ok=True)
        out_path = os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input',
                                os.path.splitext(file_name)[0],'cells')
        paths = []
        for i, row in enumerate(rows):
            for j, cell in enumerate(row):
                cell_filename = "{:03d}-{:03d}.png".format(i, j)
                path = os.path.join(out_path, cell_filename)
                cv2.imwrite(path, cell)
                paths.append(path)
        return paths

    f = os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input',
                           os.path.splitext(file_name)[0],'crop',b)
    main(f)

    def main(image_file, tess_args):
        """
        OCR the image and output the text to a file with an extension that is ready
        to be used in Tesseract training (.gt.txt).
        Tries to crop the image so that only the relevant text gets passed to Tesseract.
        Returns the name of the text file that contains the text.
        """
        #file_path = '/home/vimal/Documents/table_detect_samples/structured images/input/011364700/cells'
        #image_file = os.listdir(file_path)

        for f in image_file:
            #print(f)
            directory, filename = os.path.split(f)
            filename_sans_ext, ext = os.path.splitext(filename)
            image = cv2.imread(os.path.join(file_path,f), cv2.IMREAD_GRAYSCALE)
            cropped = crop_to_text(image)
            #ocr_data_dir = os.path.join(directory, "ocr_data")
            #os.makedirs(ocr_data_dir, exist_ok=True)
            #out_imagepath = os.path.join(ocr_data_dir, filename)
            out_path = os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input',
                                os.path.splitext(file_name)[0],'text')
            out_txtpath = os.path.join(out_path, "{}.txt".format(filename_sans_ext))
            #cv2.imwrite(out_imagepath, cropped)
            if not tess_args:
                d = os.path.dirname(sys.modules["table_ocr"].__file__)
                tessdata_dir = os.path.join(d, "tessdata")
                tess_args = ["--psm", "7", "-l", "table-ocr", "--tessdata-dir", tessdata_dir]
            txt = ocr_image(cropped, " ".join(tess_args))
            with open(out_txtpath, "w") as txt_file:
                txt_file.write(txt)
            #return out_txtpath
    def crop_to_text(image):
        MAX_COLOR_VAL = 255
        BLOCK_SIZE = 15
        SUBTRACT_FROM_MEAN = -2

        img_bin = cv2.adaptiveThreshold(
            ~image,
            MAX_COLOR_VAL,
            cv2.ADAPTIVE_THRESH_MEAN_C,
            cv2.THRESH_BINARY,
            BLOCK_SIZE,
            SUBTRACT_FROM_MEAN,
        )

        img_h, img_w = image.shape
        horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (int(img_w * 0.5), 1))
        vertical_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, int(img_h * 0.7)))
        horizontal_lines = cv2.morphologyEx(img_bin, cv2.MORPH_OPEN, horizontal_kernel)
        vertical_lines = cv2.morphologyEx(img_bin, cv2.MORPH_OPEN, vertical_kernel)
        both = horizontal_lines + vertical_lines
        cleaned = img_bin - both

        # Get rid of little noise.
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
        opened = cv2.morphologyEx(cleaned, cv2.MORPH_OPEN, kernel)
        opened = cv2.dilate(opened, kernel)

        contours, hierarchy = cv2.findContours(opened, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
        bounding_rects = [cv2.boundingRect(c) for c in contours]
        NUM_PX_COMMA = 6
        MIN_CHAR_AREA = 5 * 9
        char_sized_bounding_rects = [(x, y, w, h) for x, y, w, h in bounding_rects if w * h > MIN_CHAR_AREA]
        if char_sized_bounding_rects:
            minx, miny, maxx, maxy = math.inf, math.inf, 0, 0
            for x, y, w, h in char_sized_bounding_rects:
                minx = min(minx, x)
                miny = min(miny, y)
                maxx = max(maxx, x + w)
                maxy = max(maxy, y + h)
            x, y, w, h = minx, miny, maxx - minx, maxy - miny
            cropped = image[y:min(img_h, y+h+NUM_PX_COMMA), x:min(img_w, x+w)]
        else:
            # If we morphed out all of the text, assume an empty image.
            cropped = MAX_COLOR_VAL * np.ones(shape=(20, 100), dtype=np.uint8)
        bordered = cv2.copyMakeBorder(cropped, 5, 5, 5, 5, cv2.BORDER_CONSTANT, None, 255)
        return bordered
    def ocr_image(image, config):
        return pytesseract.image_to_string(
            image,
            lang='eng', config='--psm 6'
        )

    file_path = os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input',
                                os.path.splitext(file_name)[0],'cells')
    image_file = os.listdir(file_path)
    tess_args = os.listdir(file_path)

    main(image_file, tess_args)

    def text_files_to_csv(files):
        """Files must be sorted lexicographically
        Filenames must be <row>-<colum>.txt.
        000-000.txt
        000-001.txt
        001-000.txt
        etc...
        """
        rows = []
        for f in files:
            directory, filename = os.path.split(f)
            with open(os.path.join(file_path,f)) as of:
                txt = of.read().strip()
            row, column = map(int, filename.split(".")[0].split("-"))
            if row == len(rows):
                rows.append([])
            rows[row].append(txt)

        csv_file = io.StringIO()
        writer = csv.writer(csv_file)
        writer.writerows(rows)
        return csv_file.getvalue()

    def main(files):
        return text_files_to_csv(files)


    file_path = os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input',
                                os.path.splitext(file_name)[0],'text')
    file = os.listdir(file_path)
    files = sorted(file)

    a = main(files)
    print(a)

    c = os.path.splitext(file_name)[0]+'_text.txt'
    t = os.path.splitext(file_name)[0]+'_f-text.txt'

    #cs = os.path.splitext(file_name)[0]+'_xl.csv'

    text_file = open(os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input',
                                os.path.splitext(file_name)[0],'txt',c), "wt")
    n = text_file.write(a)
    text_file.close()
    #print(text_file)

    df = pd.read_csv(os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input',
                                os.path.splitext(file_name)[0],'txt',c),
        header=None,delimiter=',',names=list(range(10)))
    print(df)
    df1 = df[df[0].str.contains('TOTAL WAGES PAID') | df[0].str.contains('total wages paid')
                                 | df[0].str.contains('Total wages paid')]
    final = df1.dropna(axis=1)
    print(final)
    final[0] = final[0].str.replace('\d+','') 
    final[0] = final[0].str.replace(r'[^\w\s]+', '')
    final[0] = final[0].str.replace('\d+', '')
    final[0] = final[0].str.strip()
    final[1] = final[1].str.replace(r'[^\w\s]+', '')
    print(final)
    
    with open(os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input',
                                os.path.splitext(file_name)[0],'final_text',t), 'a') as f:
        f.write(final.to_string(header = False, index = False))
    text_file = open(os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input',
                                os.path.splitext(file_name)[0],'final_text',t), 'r+')
    print(text_file.read())
    
        
    

In [2]:
file_name = '0100477000.jpeg'
total_wages_paid(file_name)

"MUST HAVE AMOUNTS IN 4, 5, & 6, EVEN IF ZERO",
"4 TOTAL WAGES PAID | 339,350 47",

                                              0   1   2   3   4   5   6   7  \
0  MUST HAVE AMOUNTS IN 4, 5, & 6, EVEN IF ZERO NaN NaN NaN NaN NaN NaN NaN   
1               4 TOTAL WAGES PAID | 339,350 47 NaN NaN NaN NaN NaN NaN NaN   

    8   9  
0 NaN NaN  
1 NaN NaN  
                                 0
1  4 TOTAL WAGES PAID | 339,350 47


KeyError: 1

In [177]:
file_name = '0111295000.jpeg'
total_wages_paid(file_name)

"MUST HAVE AMOUNTS IN 4, 5, & 6, EVEN IF ZERO",
4 TOTAL WAGES PAID,glo §,

                                              0      1   2   3   4   5   6  \
0  MUST HAVE AMOUNTS IN 4, 5, & 6, EVEN IF ZERO    NaN NaN NaN NaN NaN NaN   
1                            4 TOTAL WAGES PAID  glo § NaN NaN NaN NaN NaN   

    7   8   9  
0 NaN NaN NaN  
1 NaN NaN NaN  
                    0      1
1  4 TOTAL WAGES PAID  glo §
                  0     1
1  TOTAL WAGES PAID  glo 
TOTAL WAGES PAID glo 


# GM total wages

In [11]:

def Total_wages_paid(file_name):
    import matplotlib.image as mpimg
    import matplotlib.pyplot as plt
    from PIL import Image
    import cv2
    import os
    import math
    import sys
    import numpy as np
    import pytesseract
    import io 
    import csv
    import pandas as pd
    import shutil, os

    shutil.copytree('/home/vimalkumar.s/Documents/extraction/header/total wages paid/gm/sample', 
        os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/gm',
                                 os.path.splitext(file_name)[0]))
    img = mpimg.imread(os.path.join
                       ('/home/vimalkumar.s/Documents/extraction/header/total wages paid/gm/input',
                        file_name))
    
    def extract_cell_images_from_table(image):
        BLUR_KERNEL_SIZE = (17, 17)
        STD_DEV_X_DIRECTION = 0
        STD_DEV_Y_DIRECTION = 0
        blurred = cv2.GaussianBlur(image, BLUR_KERNEL_SIZE, STD_DEV_X_DIRECTION, STD_DEV_Y_DIRECTION)
        MAX_COLOR_VAL = 255
        BLOCK_SIZE = 15
        SUBTRACT_FROM_MEAN = -2

        img_bin = cv2.adaptiveThreshold(
            ~blurred,
            MAX_COLOR_VAL,
            cv2.ADAPTIVE_THRESH_MEAN_C,
            cv2.THRESH_BINARY,
            BLOCK_SIZE,
            SUBTRACT_FROM_MEAN,
        )
        vertical = horizontal = img_bin.copy()
        SCALE = 5
        image_width, image_height = horizontal.shape
        horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (int(image_width / SCALE), 1))
        horizontally_opened = cv2.morphologyEx(img_bin, cv2.MORPH_OPEN, horizontal_kernel)
        vertical_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, int(image_height / SCALE)))
        vertically_opened = cv2.morphologyEx(img_bin, cv2.MORPH_OPEN, vertical_kernel)

        horizontally_dilated = cv2.dilate(horizontally_opened, cv2.getStructuringElement(cv2.MORPH_RECT, (40, 1)))
        vertically_dilated = cv2.dilate(vertically_opened, cv2.getStructuringElement(cv2.MORPH_RECT, (1, 60)))

        mask = horizontally_dilated + vertically_dilated
        contours, heirarchy = cv2.findContours(
            mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE,
        )

        perimeter_lengths = [cv2.arcLength(c, True) for c in contours]
        epsilons = [0.05 * p for p in perimeter_lengths]
        approx_polys = [cv2.approxPolyDP(c, e, True) for c, e in zip(contours, epsilons)]

        # Filter out contours that aren't rectangular. Those that aren't rectangular
        # are probably noise.
        approx_rects = [p for p in approx_polys if len(p) == 4]
        bounding_rects = [cv2.boundingRect(a) for a in approx_polys]

        # Filter out rectangles that are too narrow or too short.
        MIN_RECT_WIDTH = 40
        MIN_RECT_HEIGHT = 10
        bounding_rects = [
            r for r in bounding_rects if MIN_RECT_WIDTH < r[2] and MIN_RECT_HEIGHT < r[3]
        ]

        # The largest bounding rectangle is assumed to be the entire table.
        # Remove it from the list. We don't want to accidentally try to OCR
        # the entire table.
        largest_rect = max(bounding_rects, key=lambda r: r[2] * r[3])
        bounding_rects = [b for b in bounding_rects if b is not largest_rect]

        cells = [c for c in bounding_rects]
        def cell_in_same_row(c1, c2):
            c1_center = c1[1] + c1[3] - c1[3] / 2
            c2_bottom = c2[1] + c2[3]
            c2_top = c2[1]
            return c2_top < c1_center < c2_bottom

        orig_cells = [c for c in cells]
        rows = []
        while cells:
            first = cells[0]
            rest = cells[1:]
            cells_in_same_row = sorted(
                [
                    c for c in rest
                    if cell_in_same_row(c, first)
                ],
                key=lambda c: c[0]
            )

            row_cells = sorted([first] + cells_in_same_row, key=lambda c: c[0])
            rows.append(row_cells)
            cells = [
                c for c in rest
                if not cell_in_same_row(c, first)
            ]

        # Sort rows by average height of their center.
        def avg_height_of_center(row):
            centers = [y + h - h / 2 for x, y, w, h in row]
            return sum(centers) / len(centers)

        rows.sort(key=avg_height_of_center)
        cell_images_rows = []
        for row in rows:
            cell_images_row = []
            for x, y, w, h in row:
                cell_images_row.append(image[y:y+h, x:x+w])
            cell_images_rows.append(cell_images_row)
        return cell_images_rows

    def main(f):
        results = []
        directory, filename = os.path.split(f)
        table = cv2.imread(f, cv2.IMREAD_GRAYSCALE)
        rows = extract_cell_images_from_table(table)
        #cell_img_dir = os.path.join(directory, "cells")
        #os.makedirs(cell_img_dir, exist_ok=True)
        out_path = os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/gm',
                                os.path.splitext(file_name)[0],'cells')
        paths = []
        for i, row in enumerate(rows):
            for j, cell in enumerate(row):
                cell_filename = "{:03d}-{:03d}.png".format(i, j)
                path = os.path.join(out_path, cell_filename)
                cv2.imwrite(path, cell)
                paths.append(path)
        return paths

    f = os.path.join(os.path.join
                       ('/home/vimalkumar.s/Documents/extraction/header/total wages paid/gm/input',
                        file_name))
    main(f)

    def main(image_file, tess_args):
        """
        OCR the image and output the text to a file with an extension that is ready
        to be used in Tesseract training (.gt.txt).
        Tries to crop the image so that only the relevant text gets passed to Tesseract.
        Returns the name of the text file that contains the text.
        """
        #file_path = '/home/vimal/Documents/table_detect_samples/structured images/input/011364700/cells'
        #image_file = os.listdir(file_path)

        for f in image_file:
            #print(f)
            directory, filename = os.path.split(f)
            filename_sans_ext, ext = os.path.splitext(filename)
            image = cv2.imread(os.path.join(file_path,f), cv2.IMREAD_GRAYSCALE)
            cropped = crop_to_text(image)
            #ocr_data_dir = os.path.join(directory, "ocr_data")
            #os.makedirs(ocr_data_dir, exist_ok=True)
            #out_imagepath = os.path.join(ocr_data_dir, filename)
            out_path = os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/gm',
                                os.path.splitext(file_name)[0],'text')
            out_txtpath = os.path.join(out_path, "{}.txt".format(filename_sans_ext))
            #cv2.imwrite(out_imagepath, cropped)
            if not tess_args:
                d = os.path.dirname(sys.modules["table_ocr"].__file__)
                tessdata_dir = os.path.join(d, "tessdata")
                tess_args = ["--psm", "7", "-l", "table-ocr", "--tessdata-dir", tessdata_dir]
            txt = ocr_image(cropped, " ".join(tess_args))
            with open(out_txtpath, "w") as txt_file:
                txt_file.write(txt)
            #return out_txtpath
    def crop_to_text(image):
        MAX_COLOR_VAL = 255
        BLOCK_SIZE = 15
        SUBTRACT_FROM_MEAN = -2

        img_bin = cv2.adaptiveThreshold(
            ~image,
            MAX_COLOR_VAL,
            cv2.ADAPTIVE_THRESH_MEAN_C,
            cv2.THRESH_BINARY,
            BLOCK_SIZE,
            SUBTRACT_FROM_MEAN,
        )

        img_h, img_w = image.shape
        horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (int(img_w * 0.5), 1))
        vertical_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, int(img_h * 0.7)))
        horizontal_lines = cv2.morphologyEx(img_bin, cv2.MORPH_OPEN, horizontal_kernel)
        vertical_lines = cv2.morphologyEx(img_bin, cv2.MORPH_OPEN, vertical_kernel)
        both = horizontal_lines + vertical_lines
        cleaned = img_bin - both

        # Get rid of little noise.
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
        opened = cv2.morphologyEx(cleaned, cv2.MORPH_OPEN, kernel)
        opened = cv2.dilate(opened, kernel)

        contours, hierarchy = cv2.findContours(opened, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
        bounding_rects = [cv2.boundingRect(c) for c in contours]
        NUM_PX_COMMA = 6
        MIN_CHAR_AREA = 5 * 9
        char_sized_bounding_rects = [(x, y, w, h) for x, y, w, h in bounding_rects if w * h > MIN_CHAR_AREA]
        if char_sized_bounding_rects:
            minx, miny, maxx, maxy = math.inf, math.inf, 0, 0
            for x, y, w, h in char_sized_bounding_rects:
                minx = min(minx, x)
                miny = min(miny, y)
                maxx = max(maxx, x + w)
                maxy = max(maxy, y + h)
            x, y, w, h = minx, miny, maxx - minx, maxy - miny
            cropped = image[y:min(img_h, y+h+NUM_PX_COMMA), x:min(img_w, x+w)]
        else:
            # If we morphed out all of the text, assume an empty image.
            cropped = MAX_COLOR_VAL * np.ones(shape=(20, 100), dtype=np.uint8)
        bordered = cv2.copyMakeBorder(cropped, 5, 5, 5, 5, cv2.BORDER_CONSTANT, None, 255)
        return bordered
    def ocr_image(image, config):
        return pytesseract.image_to_string(
            image,
            lang='eng', config='--psm 6'
        )

    file_path = os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/gm',
                                os.path.splitext(file_name)[0],'cells')
    image_file = os.listdir(file_path)
    tess_args = os.listdir(file_path)

    main(image_file, tess_args)

    def text_files_to_csv(files):
        """Files must be sorted lexicographically
        Filenames must be <row>-<colum>.txt.
        000-000.txt
        000-001.txt
        001-000.txt
        etc...
        """
        rows = []
        for f in files:
            directory, filename = os.path.split(f)
            with open(os.path.join(file_path,f)) as of:
                txt = of.read().strip()
            row, column = map(int, filename.split(".")[0].split("-"))
            if row == len(rows):
                rows.append([])
            rows[row].append(txt)

        csv_file = io.StringIO()
        writer = csv.writer(csv_file)
        writer.writerows(rows)
        return csv_file.getvalue()

    def main(files):
        return text_files_to_csv(files)


    file_path = os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/gm',
                                os.path.splitext(file_name)[0],'text')
    file = os.listdir(file_path)
    files = sorted(file)

    a = main(files)
    #print(a)

    c = os.path.splitext(file_name)[0]+'_text.txt'
    #cs = os.path.splitext(file_name)[0]+'_xl.csv'

    text_file = open(os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/gm',
                                os.path.splitext(file_name)[0],'txt',c), "wt")
    n = text_file.write(a)
    text_file.close()

    df = pd.read_csv(os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/gm',
                                os.path.splitext(file_name)[0],'txt',c),
        header=None,delimiter=',',names=list(range(10)))
    df1 = df[df[0].str.contains('TOTAL WAGES PAID') | df[0].str.contains('total wages paid')
                                 | df[0].str.contains('Total wages paid')]
    final = df1.dropna(axis=1)
    print(final[final.columns[0:2]])

In [12]:
file_name = '0534088000.jpeg'
Total_wages_paid(file_name)

                    0          1
3  4 TOTAL WAGES PAID  64,571.94


In [3]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from PIL import Image
import cv2
import os
import math
import sys
import numpy as np
import pytesseract
import io 
import csv
import pandas as pd
import shutil, os
import warnings
warnings.filterwarnings('ignore')
    
path = '/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input/input'
for file_name in os.listdir(path):
    #print(file_name)
    shutil.copytree('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input/sample', 
        os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input',
                                 os.path.splitext(file_name)[0]))
     
    img = mpimg.imread(os.path.join
                       ('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input/input',
                        file_name))
    width = img.shape[1]
    height = img.shape[0]
    wi = width * 0.95
    he = height * 0.3
    w = int(wi)
    h = int(he)
    y = 0
    x1 = w * 0.45
    x = int(x1)
    crop = img[y:y+h, x:x+w]
    #plt.imshow(crop)
    im = Image.fromarray(crop)
    b = os.path.splitext(file_name)[0]+'_crop.jpeg'
    im.save(os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input',
                           os.path.splitext(file_name)[0],'crop',b))
    def extract_cell_images_from_table(image):
        BLUR_KERNEL_SIZE = (17, 17)
        STD_DEV_X_DIRECTION = 0
        STD_DEV_Y_DIRECTION = 0
        blurred = cv2.GaussianBlur(image, BLUR_KERNEL_SIZE, STD_DEV_X_DIRECTION, STD_DEV_Y_DIRECTION)
        MAX_COLOR_VAL = 255
        BLOCK_SIZE = 15
        SUBTRACT_FROM_MEAN = -2

        img_bin = cv2.adaptiveThreshold(
            ~blurred,
            MAX_COLOR_VAL,
            cv2.ADAPTIVE_THRESH_MEAN_C,
            cv2.THRESH_BINARY,
            BLOCK_SIZE,
            SUBTRACT_FROM_MEAN,
        )
        vertical = horizontal = img_bin.copy()
        SCALE = 5
        image_width, image_height = horizontal.shape
        horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (int(image_width / SCALE), 1))
        horizontally_opened = cv2.morphologyEx(img_bin, cv2.MORPH_OPEN, horizontal_kernel)
        vertical_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, int(image_height / SCALE)))
        vertically_opened = cv2.morphologyEx(img_bin, cv2.MORPH_OPEN, vertical_kernel)

        horizontally_dilated = cv2.dilate(horizontally_opened, cv2.getStructuringElement(cv2.MORPH_RECT, (40, 1)))
        vertically_dilated = cv2.dilate(vertically_opened, cv2.getStructuringElement(cv2.MORPH_RECT, (1, 60)))

        mask = horizontally_dilated + vertically_dilated
        contours, heirarchy = cv2.findContours(
            mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE,
        )

        perimeter_lengths = [cv2.arcLength(c, True) for c in contours]
        epsilons = [0.05 * p for p in perimeter_lengths]
        approx_polys = [cv2.approxPolyDP(c, e, True) for c, e in zip(contours, epsilons)]

        # Filter out contours that aren't rectangular. Those that aren't rectangular
        # are probably noise.
        approx_rects = [p for p in approx_polys if len(p) == 4]
        bounding_rects = [cv2.boundingRect(a) for a in approx_polys]

        # Filter out rectangles that are too narrow or too short.
        MIN_RECT_WIDTH = 40
        MIN_RECT_HEIGHT = 10
        bounding_rects = [
            r for r in bounding_rects if MIN_RECT_WIDTH < r[2] and MIN_RECT_HEIGHT < r[3]
        ]

        # The largest bounding rectangle is assumed to be the entire table.
        # Remove it from the list. We don't want to accidentally try to OCR
        # the entire table.
        largest_rect = max(bounding_rects, key=lambda r: r[2] * r[3])
        bounding_rects = [b for b in bounding_rects if b is not largest_rect]

        cells = [c for c in bounding_rects]
        def cell_in_same_row(c1, c2):
            c1_center = c1[1] + c1[3] - c1[3] / 2
            c2_bottom = c2[1] + c2[3]
            c2_top = c2[1]
            return c2_top < c1_center < c2_bottom

        orig_cells = [c for c in cells]
        rows = []
        while cells:
            first = cells[0]
            rest = cells[1:]
            cells_in_same_row = sorted(
                [
                    c for c in rest
                    if cell_in_same_row(c, first)
                ],
                key=lambda c: c[0]
            )

            row_cells = sorted([first] + cells_in_same_row, key=lambda c: c[0])
            rows.append(row_cells)
            cells = [
                c for c in rest
                if not cell_in_same_row(c, first)
            ]

        # Sort rows by average height of their center.
        def avg_height_of_center(row):
            centers = [y + h - h / 2 for x, y, w, h in row]
            return sum(centers) / len(centers)

        rows.sort(key=avg_height_of_center)
        cell_images_rows = []
        for row in rows:
            cell_images_row = []
            for x, y, w, h in row:
                cell_images_row.append(image[y:y+h, x:x+w])
            cell_images_rows.append(cell_images_row)
        return cell_images_rows

    def main(f):
        results = []
        directory, filename = os.path.split(f)
        table = cv2.imread(f, cv2.IMREAD_GRAYSCALE)
        rows = extract_cell_images_from_table(table)
        #cell_img_dir = os.path.join(directory, "cells")
        #os.makedirs(cell_img_dir, exist_ok=True)
        out_path = os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input',
                                os.path.splitext(file_name)[0],'cells')
        paths = []
        for i, row in enumerate(rows):
            for j, cell in enumerate(row):
                cell_filename = "{:03d}-{:03d}.png".format(i, j)
                path = os.path.join(out_path, cell_filename)
                cv2.imwrite(path, cell)
                paths.append(path)
        return paths

    f = os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input',
                           os.path.splitext(file_name)[0],'crop',b)
    main(f)
    
    def main(image_file, tess_args):
        """
        OCR the image and output the text to a file with an extension that is ready
        to be used in Tesseract training (.gt.txt).
        Tries to crop the image so that only the relevant text gets passed to Tesseract.
        Returns the name of the text file that contains the text.
        """
        #file_path = '/home/vimal/Documents/table_detect_samples/structured images/input/011364700/cells'
        #image_file = os.listdir(file_path)

        for f in image_file:
            #print(f)
            directory, filename = os.path.split(f)
            filename_sans_ext, ext = os.path.splitext(filename)
            image = cv2.imread(os.path.join(file_path,f), cv2.IMREAD_GRAYSCALE)
            cropped = crop_to_text(image)
            #ocr_data_dir = os.path.join(directory, "ocr_data")
            #os.makedirs(ocr_data_dir, exist_ok=True)
            #out_imagepath = os.path.join(ocr_data_dir, filename)
            out_path = os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input',
                                os.path.splitext(file_name)[0],'text')
            out_txtpath = os.path.join(out_path, "{}.txt".format(filename_sans_ext))
            #cv2.imwrite(out_imagepath, cropped)
            if not tess_args:
                d = os.path.dirname(sys.modules["table_ocr"].__file__)
                tessdata_dir = os.path.join(d, "tessdata")
                tess_args = ["--psm", "7", "-l", "table-ocr", "--tessdata-dir", tessdata_dir]
            txt = ocr_image(cropped, " ".join(tess_args))
            with open(out_txtpath, "w") as txt_file:
                txt_file.write(txt)
            #return out_txtpath
    def crop_to_text(image):
        MAX_COLOR_VAL = 255
        BLOCK_SIZE = 15
        SUBTRACT_FROM_MEAN = -2

        img_bin = cv2.adaptiveThreshold(
            ~image,
            MAX_COLOR_VAL,
            cv2.ADAPTIVE_THRESH_MEAN_C,
            cv2.THRESH_BINARY,
            BLOCK_SIZE,
            SUBTRACT_FROM_MEAN,
        )

        img_h, img_w = image.shape
        horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (int(img_w * 0.5), 1))
        vertical_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, int(img_h * 0.7)))
        horizontal_lines = cv2.morphologyEx(img_bin, cv2.MORPH_OPEN, horizontal_kernel)
        vertical_lines = cv2.morphologyEx(img_bin, cv2.MORPH_OPEN, vertical_kernel)
        both = horizontal_lines + vertical_lines
        cleaned = img_bin - both

        # Get rid of little noise.
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
        opened = cv2.morphologyEx(cleaned, cv2.MORPH_OPEN, kernel)
        opened = cv2.dilate(opened, kernel)

        contours, hierarchy = cv2.findContours(opened, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
        bounding_rects = [cv2.boundingRect(c) for c in contours]
        NUM_PX_COMMA = 6
        MIN_CHAR_AREA = 5 * 9
        char_sized_bounding_rects = [(x, y, w, h) for x, y, w, h in bounding_rects if w * h > MIN_CHAR_AREA]
        if char_sized_bounding_rects:
            minx, miny, maxx, maxy = math.inf, math.inf, 0, 0
            for x, y, w, h in char_sized_bounding_rects:
                minx = min(minx, x)
                miny = min(miny, y)
                maxx = max(maxx, x + w)
                maxy = max(maxy, y + h)
            x, y, w, h = minx, miny, maxx - minx, maxy - miny
            cropped = image[y:min(img_h, y+h+NUM_PX_COMMA), x:min(img_w, x+w)]
        else:
            # If we morphed out all of the text, assume an empty image.
            cropped = MAX_COLOR_VAL * np.ones(shape=(20, 100), dtype=np.uint8)
        bordered = cv2.copyMakeBorder(cropped, 5, 5, 5, 5, cv2.BORDER_CONSTANT, None, 255)
        return bordered
    def ocr_image(image, config):
        return pytesseract.image_to_string(
            image,
            lang='eng', config='--psm 6'
        )

    file_path = os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input',
                                os.path.splitext(file_name)[0],'cells')
    image_file = os.listdir(file_path)
    tess_args = os.listdir(file_path)

    main(image_file, tess_args)
    
    def text_files_to_csv(files):
        """Files must be sorted lexicographically
        Filenames must be <row>-<colum>.txt.
        000-000.txt
        000-001.txt
        001-000.txt
        etc...
        """
        rows = []
        for f in files:
            directory, filename = os.path.split(f)
            with open(os.path.join(file_path,f)) as of:
                txt = of.read().strip()
            row, column = map(int, filename.split(".")[0].split("-"))
            if row == len(rows):
                rows.append([])
            rows[row].append(txt)

        csv_file = io.StringIO()
        writer = csv.writer(csv_file)
        writer.writerows(rows)
        return csv_file.getvalue()

    def main(files):
        return text_files_to_csv(files)


    file_path = os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input',
                                os.path.splitext(file_name)[0],'text')
    file = os.listdir(file_path)
    files = sorted(file)

    a = main(files)
    #print(a)

    c = os.path.splitext(file_name)[0]+'_text.txt'
    t = os.path.splitext(file_name)[0]+'_f-text.txt'

    #cs = os.path.splitext(file_name)[0]+'_xl.csv'

    text_file = open(os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input',
                                os.path.splitext(file_name)[0],'txt',c), "wt")
    n = text_file.write(a)
    text_file.close()

    df = pd.read_csv(os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input',
                                os.path.splitext(file_name)[0],'txt',c),
        header=None,delimiter=',',names=list(range(10)))
    
    if df[0].dtypes != 'object':
        print(os.path.splitext(file_name)[0]+'.tif','TOTAL WAGES PAID',0)
    else:
        df1 = df[df[0].str.contains('TOTAL WAGES PAID') | df[0].str.contains('total wages paid')
                                 | df[0].str.contains('Total wages paid')]
        final = df1.dropna(axis=1)
        final = final.astype(str)
        #print(final)
        final[0] = final[0].str.replace('\d+','') 
        final[0] = final[0].str.replace(r'[^\w\s]+', '')
        final[0] = final[0].str.replace('\d+', '')
        final[0] = final[0].str.strip()
        
        if final.shape[1]>1:
            final[1] = final[1].str.replace(r'[^\w\s]+', '')
            final[1] = final[1].str.replace(" ","")

        #print(final)
        if len(final) == 0:
            print(os.path.splitext(file_name)[0]+'.tif','TOTAL WAGES PAID',0)
        else:       
            with open(os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input',
                                os.path.splitext(file_name)[0],'final_text',t), 'a') as f:
                f.write(final.to_string(header = False, index = False))
            text_file = open(os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input',
                                os.path.splitext(file_name)[0],'final_text',t), 'r+')
            print(os.path.splitext(file_name)[0]+'.tif',text_file.read())

0109374000.tif TOTAL WAGES PAID 0
0497990000.tif TOTAL WAGES PAID 249400
0496106000.tif TOTAL WAGES PAID 19035118
0103397000.tif TOTAL WAGES PAID 663517233
0110228000.tif TOTAL WAGES PAID 8298131
0500847001.tif TOTAL WAGES PAID 90754
0107313000.tif TOTAL WAGES PAID 4079264
0104702000.tif TOTAL WAGES PAID 0
0102281000.tif TOTAL WAGES PAID 7786950
0496813000.tif TOTAL WAGES PAID 3163933
0108918000.tif TOTAL WAGES PAID 0
0499639000.tif TOTAL WAGES PAID 12854493
0496350000.tif TOTAL WAGES PAID 2190239
0498112000.tif TOTAL WAGES PAID 5419282
0110331000.tif TOTAL WAGES PAID 3777105407
0504031000.tif TOTAL WAGES PAID 18805150
0110040000.tif TOTAL WAGES PAID 747145
0102880000.tif TOTAL WAGES PAID 114370139
0503866000.tif TOTAL WAGES PAID 11421015
0104921000.tif TOTAL WAGES PAID 25642596
0109337000.tif TOTAL WAGES PAID 9758340
0499335000.tif TOTAL WAGES PAID 0
0102551000.tif TOTAL WAGES PAID 01553224
0105311000.tif TOTAL WAGES PAID 11234600
0504111000.tif TOTAL WAGES PAID 1715246
0500221000.tif

In [261]:
def twp(file_name):
    import matplotlib.image as mpimg
    import matplotlib.pyplot as plt
    from PIL import Image
    import cv2
    import os
    import math
    import sys
    import numpy as np
    import pytesseract
    import io 
    import csv
    import pandas as pd
    import shutil, os
    import warnings
    warnings.filterwarnings('ignore')
    
    #file_name = '0100477000.jpeg'

    #path = '/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input/input'
    #for file_name in os.listdir(path):
        #print(file_name)
    shutil.copytree('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input/sample', 
        os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input',
                                 os.path.splitext(file_name)[0]))
     
    img = mpimg.imread(os.path.join
                       ('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input/input',
                        file_name))
    width = img.shape[1]
    height = img.shape[0]
    wi = width * 0.95
    he = height * 0.3
    w = int(wi)
    h = int(he)
    y = 0
    x1 = w * 0.45
    x = int(x1)
    crop = img[y:y+h, x:x+w]
    #plt.imshow(crop)
    im = Image.fromarray(crop)
    b = os.path.splitext(file_name)[0]+'_crop.jpeg'
    im.save(os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input',
                           os.path.splitext(file_name)[0],'crop',b))
    def extract_cell_images_from_table(image):
        BLUR_KERNEL_SIZE = (17, 17)
        STD_DEV_X_DIRECTION = 0
        STD_DEV_Y_DIRECTION = 0
        blurred = cv2.GaussianBlur(image, BLUR_KERNEL_SIZE, STD_DEV_X_DIRECTION, STD_DEV_Y_DIRECTION)
        MAX_COLOR_VAL = 255
        BLOCK_SIZE = 15
        SUBTRACT_FROM_MEAN = -2

        img_bin = cv2.adaptiveThreshold(
            ~blurred,
            MAX_COLOR_VAL,
            cv2.ADAPTIVE_THRESH_MEAN_C,
            cv2.THRESH_BINARY,
            BLOCK_SIZE,
            SUBTRACT_FROM_MEAN,
        )
        vertical = horizontal = img_bin.copy()
        SCALE = 5
        image_width, image_height = horizontal.shape
        horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (int(image_width / SCALE), 1))
        horizontally_opened = cv2.morphologyEx(img_bin, cv2.MORPH_OPEN, horizontal_kernel)
        vertical_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, int(image_height / SCALE)))
        vertically_opened = cv2.morphologyEx(img_bin, cv2.MORPH_OPEN, vertical_kernel)

        horizontally_dilated = cv2.dilate(horizontally_opened, cv2.getStructuringElement(cv2.MORPH_RECT, (40, 1)))
        vertically_dilated = cv2.dilate(vertically_opened, cv2.getStructuringElement(cv2.MORPH_RECT, (1, 60)))

        mask = horizontally_dilated + vertically_dilated
        contours, heirarchy = cv2.findContours(
            mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE,
        )

        perimeter_lengths = [cv2.arcLength(c, True) for c in contours]
        epsilons = [0.05 * p for p in perimeter_lengths]
        approx_polys = [cv2.approxPolyDP(c, e, True) for c, e in zip(contours, epsilons)]

        # Filter out contours that aren't rectangular. Those that aren't rectangular
        # are probably noise.
        approx_rects = [p for p in approx_polys if len(p) == 4]
        bounding_rects = [cv2.boundingRect(a) for a in approx_polys]

        # Filter out rectangles that are too narrow or too short.
        MIN_RECT_WIDTH = 40
        MIN_RECT_HEIGHT = 10
        bounding_rects = [
            r for r in bounding_rects if MIN_RECT_WIDTH < r[2] and MIN_RECT_HEIGHT < r[3]
        ]

        # The largest bounding rectangle is assumed to be the entire table.
        # Remove it from the list. We don't want to accidentally try to OCR
        # the entire table.
        largest_rect = max(bounding_rects, key=lambda r: r[2] * r[3])
        bounding_rects = [b for b in bounding_rects if b is not largest_rect]

        cells = [c for c in bounding_rects]
        def cell_in_same_row(c1, c2):
            c1_center = c1[1] + c1[3] - c1[3] / 2
            c2_bottom = c2[1] + c2[3]
            c2_top = c2[1]
            return c2_top < c1_center < c2_bottom

        orig_cells = [c for c in cells]
        rows = []
        while cells:
            first = cells[0]
            rest = cells[1:]
            cells_in_same_row = sorted(
                [
                    c for c in rest
                    if cell_in_same_row(c, first)
                ],
                key=lambda c: c[0]
            )

            row_cells = sorted([first] + cells_in_same_row, key=lambda c: c[0])
            rows.append(row_cells)
            cells = [
                c for c in rest
                if not cell_in_same_row(c, first)
            ]

        # Sort rows by average height of their center.
        def avg_height_of_center(row):
            centers = [y + h - h / 2 for x, y, w, h in row]
            return sum(centers) / len(centers)

        rows.sort(key=avg_height_of_center)
        cell_images_rows = []
        for row in rows:
            cell_images_row = []
            for x, y, w, h in row:
                cell_images_row.append(image[y:y+h, x:x+w])
            cell_images_rows.append(cell_images_row)
        return cell_images_rows

    def main(f):
        results = []
        directory, filename = os.path.split(f)
        table = cv2.imread(f, cv2.IMREAD_GRAYSCALE)
        rows = extract_cell_images_from_table(table)
        #cell_img_dir = os.path.join(directory, "cells")
        #os.makedirs(cell_img_dir, exist_ok=True)
        out_path = os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input',
                                os.path.splitext(file_name)[0],'cells')
        paths = []
        for i, row in enumerate(rows):
            for j, cell in enumerate(row):
                cell_filename = "{:03d}-{:03d}.png".format(i, j)
                path = os.path.join(out_path, cell_filename)
                cv2.imwrite(path, cell)
                paths.append(path)
        return paths

    f = os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input',
                           os.path.splitext(file_name)[0],'crop',b)
    main(f)
    
    def main(image_file, tess_args):
        """
        OCR the image and output the text to a file with an extension that is ready
        to be used in Tesseract training (.gt.txt).
        Tries to crop the image so that only the relevant text gets passed to Tesseract.
        Returns the name of the text file that contains the text.
        """
        #file_path = '/home/vimal/Documents/table_detect_samples/structured images/input/011364700/cells'
        #image_file = os.listdir(file_path)

        for f in image_file:
            #print(f)
            directory, filename = os.path.split(f)
            filename_sans_ext, ext = os.path.splitext(filename)
            image = cv2.imread(os.path.join(file_path,f), cv2.IMREAD_GRAYSCALE)
            cropped = crop_to_text(image)
            #ocr_data_dir = os.path.join(directory, "ocr_data")
            #os.makedirs(ocr_data_dir, exist_ok=True)
            #out_imagepath = os.path.join(ocr_data_dir, filename)
            out_path = os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input',
                                os.path.splitext(file_name)[0],'text')
            out_txtpath = os.path.join(out_path, "{}.txt".format(filename_sans_ext))
            #cv2.imwrite(out_imagepath, cropped)
            if not tess_args:
                d = os.path.dirname(sys.modules["table_ocr"].__file__)
                tessdata_dir = os.path.join(d, "tessdata")
                tess_args = ["--psm", "7", "-l", "table-ocr", "--tessdata-dir", tessdata_dir]
            txt = ocr_image(cropped, " ".join(tess_args))
            with open(out_txtpath, "w") as txt_file:
                txt_file.write(txt)
            #return out_txtpath
    def crop_to_text(image):
        MAX_COLOR_VAL = 255
        BLOCK_SIZE = 15
        SUBTRACT_FROM_MEAN = -2

        img_bin = cv2.adaptiveThreshold(
            ~image,
            MAX_COLOR_VAL,
            cv2.ADAPTIVE_THRESH_MEAN_C,
            cv2.THRESH_BINARY,
            BLOCK_SIZE,
            SUBTRACT_FROM_MEAN,
        )

        img_h, img_w = image.shape
        horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (int(img_w * 0.5), 1))
        vertical_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, int(img_h * 0.7)))
        horizontal_lines = cv2.morphologyEx(img_bin, cv2.MORPH_OPEN, horizontal_kernel)
        vertical_lines = cv2.morphologyEx(img_bin, cv2.MORPH_OPEN, vertical_kernel)
        both = horizontal_lines + vertical_lines
        cleaned = img_bin - both

        # Get rid of little noise.
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
        opened = cv2.morphologyEx(cleaned, cv2.MORPH_OPEN, kernel)
        opened = cv2.dilate(opened, kernel)

        contours, hierarchy = cv2.findContours(opened, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
        bounding_rects = [cv2.boundingRect(c) for c in contours]
        NUM_PX_COMMA = 6
        MIN_CHAR_AREA = 5 * 9
        char_sized_bounding_rects = [(x, y, w, h) for x, y, w, h in bounding_rects if w * h > MIN_CHAR_AREA]
        if char_sized_bounding_rects:
            minx, miny, maxx, maxy = math.inf, math.inf, 0, 0
            for x, y, w, h in char_sized_bounding_rects:
                minx = min(minx, x)
                miny = min(miny, y)
                maxx = max(maxx, x + w)
                maxy = max(maxy, y + h)
            x, y, w, h = minx, miny, maxx - minx, maxy - miny
            cropped = image[y:min(img_h, y+h+NUM_PX_COMMA), x:min(img_w, x+w)]
        else:
            # If we morphed out all of the text, assume an empty image.
            cropped = MAX_COLOR_VAL * np.ones(shape=(20, 100), dtype=np.uint8)
        bordered = cv2.copyMakeBorder(cropped, 5, 5, 5, 5, cv2.BORDER_CONSTANT, None, 255)
        return bordered
    def ocr_image(image, config):
        return pytesseract.image_to_string(
            image,
            lang='eng', config='--psm 6'
        )

    file_path = os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input',
                                os.path.splitext(file_name)[0],'cells')
    image_file = os.listdir(file_path)
    tess_args = os.listdir(file_path)

    main(image_file, tess_args)
    
    def text_files_to_csv(files):
        """Files must be sorted lexicographically
        Filenames must be <row>-<colum>.txt.
        000-000.txt
        000-001.txt
        001-000.txt
        etc...
        """
        rows = []
        for f in files:
            directory, filename = os.path.split(f)
            with open(os.path.join(file_path,f)) as of:
                txt = of.read().strip()
            row, column = map(int, filename.split(".")[0].split("-"))
            if row == len(rows):
                rows.append([])
            rows[row].append(txt)

        csv_file = io.StringIO()
        writer = csv.writer(csv_file)
        writer.writerows(rows)
        return csv_file.getvalue()

    def main(files):
        return text_files_to_csv(files)


    file_path = os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input',
                                os.path.splitext(file_name)[0],'text')
    file = os.listdir(file_path)
    files = sorted(file)

    a = main(files)
    #print(a)

    c = os.path.splitext(file_name)[0]+'_text.txt'
    t = os.path.splitext(file_name)[0]+'_f-text.txt'

    #cs = os.path.splitext(file_name)[0]+'_xl.csv'

    text_file = open(os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input',
                                os.path.splitext(file_name)[0],'txt',c), "wt")
    n = text_file.write(a)
    text_file.close()

    df = pd.read_csv(os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input',
                                os.path.splitext(file_name)[0],'txt',c),
        header=None,delimiter=',',names=list(range(10)))
    #print(df)
    
    df1 = df.dropna(axis = 0,how = 'all')
    #print(df1)
    
    word = 'TOTAL WAGES PAID'
    
        
    
    if df1[0].dtypes != 'object':
        print(os.path.splitext(file_name)[0]+'.tif','TOTAL WAGES PAID',0)
        
    else:
        
        for i in df1[0]:
            if fuzz.ratio(word,i) > 80:
                #print(i)
                dff = df1[df1[0] == i]
                dff = dff.dropna(axis = 1,how = 'all')
                output = dff[1]
                output = output.str.replace(r'[^\w\s]+', '')
                output = output.str.replace(" ","")
                #print(output)
                if len(output) == 0:
                    print(os.path.splitext(file_name)[0]+'.tif','TOTAL WAGES PAID',0)
                else:
                    with open(os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input',
                                            os.path.splitext(file_name)[0],'final_text',t), 'a') as f:
                        f.write(output.to_string(header = False, index = False))
                    text_file = open(os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input',
                                            os.path.splitext(file_name)[0],'final_text',t), 'r+')
                    print(os.path.splitext(file_name)[0]+'.tif','TOTAL WAGES PAID',text_file.read())
        
                
                
    
            
        

In [263]:
file_name = '0497041000.jpeg'
twp(file_name)

0497041000.tif TOTAL WAGES PAID 1150000


In [58]:
    import matplotlib.image as mpimg
    import matplotlib.pyplot as plt
    from PIL import Image
    import cv2
    import os
    import math
    import sys
    import numpy as np
    import pytesseract
    import io 
    import csv
    import pandas as pd
    import re
    import shutil, os
    from fuzzywuzzy import fuzz
    from fuzzywuzzy import process
    import warnings
    warnings.filterwarnings('ignore')

    file_name = '0500847001.jpeg'

#path = '/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input/input'
#for file_name in os.listdir(path):
    #print(file_name)
    shutil.copytree('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input/sample', 
        os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input',
                                 os.path.splitext(file_name)[0]))
     
    img = mpimg.imread(os.path.join
                       ('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input/input',
                        file_name))
    width = img.shape[1]
    height = img.shape[0]
    wi = width * 0.95
    he = height * 0.3
    w = int(wi)
    h = int(he)
    y = 0
    x1 = w * 0.45
    x = int(x1)
    crop = img[y:y+h, x:x+w]
    #plt.imshow(crop)
    im = Image.fromarray(crop)
    b = os.path.splitext(file_name)[0]+'_crop.jpeg'
    im.save(os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input',
                           os.path.splitext(file_name)[0],'crop',b))
    def extract_cell_images_from_table(image):
        BLUR_KERNEL_SIZE = (17, 17)
        STD_DEV_X_DIRECTION = 0
        STD_DEV_Y_DIRECTION = 0
        blurred = cv2.GaussianBlur(image, BLUR_KERNEL_SIZE, STD_DEV_X_DIRECTION, STD_DEV_Y_DIRECTION)
        MAX_COLOR_VAL = 255
        BLOCK_SIZE = 15
        SUBTRACT_FROM_MEAN = -2

        img_bin = cv2.adaptiveThreshold(
            ~blurred,
            MAX_COLOR_VAL,
            cv2.ADAPTIVE_THRESH_MEAN_C,
            cv2.THRESH_BINARY,
            BLOCK_SIZE,
            SUBTRACT_FROM_MEAN,
        )
        vertical = horizontal = img_bin.copy()
        SCALE = 5
        image_width, image_height = horizontal.shape
        horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (int(image_width / SCALE), 1))
        horizontally_opened = cv2.morphologyEx(img_bin, cv2.MORPH_OPEN, horizontal_kernel)
        vertical_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, int(image_height / SCALE)))
        vertically_opened = cv2.morphologyEx(img_bin, cv2.MORPH_OPEN, vertical_kernel)

        horizontally_dilated = cv2.dilate(horizontally_opened, cv2.getStructuringElement(cv2.MORPH_RECT, (40, 1)))
        vertically_dilated = cv2.dilate(vertically_opened, cv2.getStructuringElement(cv2.MORPH_RECT, (1, 60)))

        mask = horizontally_dilated + vertically_dilated
        contours, heirarchy = cv2.findContours(
            mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE,
        )

        perimeter_lengths = [cv2.arcLength(c, True) for c in contours]
        epsilons = [0.05 * p for p in perimeter_lengths]
        approx_polys = [cv2.approxPolyDP(c, e, True) for c, e in zip(contours, epsilons)]

        # Filter out contours that aren't rectangular. Those that aren't rectangular
        # are probably noise.
        approx_rects = [p for p in approx_polys if len(p) == 4]
        bounding_rects = [cv2.boundingRect(a) for a in approx_polys]

        # Filter out rectangles that are too narrow or too short.
        MIN_RECT_WIDTH = 40
        MIN_RECT_HEIGHT = 10
        bounding_rects = [
            r for r in bounding_rects if MIN_RECT_WIDTH < r[2] and MIN_RECT_HEIGHT < r[3]
        ]

        # The largest bounding rectangle is assumed to be the entire table.
        # Remove it from the list. We don't want to accidentally try to OCR
        # the entire table.
        largest_rect = max(bounding_rects, key=lambda r: r[2] * r[3])
        bounding_rects = [b for b in bounding_rects if b is not largest_rect]

        cells = [c for c in bounding_rects]
        def cell_in_same_row(c1, c2):
            c1_center = c1[1] + c1[3] - c1[3] / 2
            c2_bottom = c2[1] + c2[3]
            c2_top = c2[1]
            return c2_top < c1_center < c2_bottom

        orig_cells = [c for c in cells]
        rows = []
        while cells:
            first = cells[0]
            rest = cells[1:]
            cells_in_same_row = sorted(
                [
                    c for c in rest
                    if cell_in_same_row(c, first)
                ],
                key=lambda c: c[0]
            )

            row_cells = sorted([first] + cells_in_same_row, key=lambda c: c[0])
            rows.append(row_cells)
            cells = [
                c for c in rest
                if not cell_in_same_row(c, first)
            ]

        # Sort rows by average height of their center.
        def avg_height_of_center(row):
            centers = [y + h - h / 2 for x, y, w, h in row]
            return sum(centers) / len(centers)

        rows.sort(key=avg_height_of_center)
        cell_images_rows = []
        for row in rows:
            cell_images_row = []
            for x, y, w, h in row:
                cell_images_row.append(image[y:y+h, x:x+w])
            cell_images_rows.append(cell_images_row)
        return cell_images_rows

    def main(f):
        results = []
        directory, filename = os.path.split(f)
        table = cv2.imread(f, cv2.IMREAD_GRAYSCALE)
        rows = extract_cell_images_from_table(table)
        #cell_img_dir = os.path.join(directory, "cells")
        #os.makedirs(cell_img_dir, exist_ok=True)
        out_path = os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input',
                                os.path.splitext(file_name)[0],'cells')
        paths = []
        for i, row in enumerate(rows):
            for j, cell in enumerate(row):
                cell_filename = "{:03d}-{:03d}.png".format(i, j)
                path = os.path.join(out_path, cell_filename)
                cv2.imwrite(path, cell)
                paths.append(path)
        return paths

    f = os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input',
                           os.path.splitext(file_name)[0],'crop',b)
    main(f)
    
    def main(image_file, tess_args):
        """
        OCR the image and output the text to a file with an extension that is ready
        to be used in Tesseract training (.gt.txt).
        Tries to crop the image so that only the relevant text gets passed to Tesseract.
        Returns the name of the text file that contains the text.
        """
        #file_path = '/home/vimal/Documents/table_detect_samples/structured images/input/011364700/cells'
        #image_file = os.listdir(file_path)

        for f in image_file:
            #print(f)
            directory, filename = os.path.split(f)
            filename_sans_ext, ext = os.path.splitext(filename)
            image = cv2.imread(os.path.join(file_path,f), cv2.IMREAD_GRAYSCALE)
            cropped = crop_to_text(image)
            #ocr_data_dir = os.path.join(directory, "ocr_data")
            #os.makedirs(ocr_data_dir, exist_ok=True)
            #out_imagepath = os.path.join(ocr_data_dir, filename)
            out_path = os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input',
                                os.path.splitext(file_name)[0],'text')
            out_txtpath = os.path.join(out_path, "{}.txt".format(filename_sans_ext))
            #cv2.imwrite(out_imagepath, cropped)
            if not tess_args:
                d = os.path.dirname(sys.modules["table_ocr"].__file__)
                tessdata_dir = os.path.join(d, "tessdata")
                tess_args = ["--psm", "7", "-l", "table-ocr", "--tessdata-dir", tessdata_dir]
            txt = ocr_image(cropped, " ".join(tess_args))
            with open(out_txtpath, "w") as txt_file:
                txt_file.write(txt)
            #return out_txtpath
    def crop_to_text(image):
        MAX_COLOR_VAL = 255
        BLOCK_SIZE = 15
        SUBTRACT_FROM_MEAN = -2

        img_bin = cv2.adaptiveThreshold(
            ~image,
            MAX_COLOR_VAL,
            cv2.ADAPTIVE_THRESH_MEAN_C,
            cv2.THRESH_BINARY,
            BLOCK_SIZE,
            SUBTRACT_FROM_MEAN,
        )

        img_h, img_w = image.shape
        horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (int(img_w * 0.5), 1))
        vertical_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, int(img_h * 0.7)))
        horizontal_lines = cv2.morphologyEx(img_bin, cv2.MORPH_OPEN, horizontal_kernel)
        vertical_lines = cv2.morphologyEx(img_bin, cv2.MORPH_OPEN, vertical_kernel)
        both = horizontal_lines + vertical_lines
        cleaned = img_bin - both

        # Get rid of little noise.
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
        opened = cv2.morphologyEx(cleaned, cv2.MORPH_OPEN, kernel)
        opened = cv2.dilate(opened, kernel)

        contours, hierarchy = cv2.findContours(opened, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
        bounding_rects = [cv2.boundingRect(c) for c in contours]
        NUM_PX_COMMA = 6
        MIN_CHAR_AREA = 5 * 9
        char_sized_bounding_rects = [(x, y, w, h) for x, y, w, h in bounding_rects if w * h > MIN_CHAR_AREA]
        if char_sized_bounding_rects:
            minx, miny, maxx, maxy = math.inf, math.inf, 0, 0
            for x, y, w, h in char_sized_bounding_rects:
                minx = min(minx, x)
                miny = min(miny, y)
                maxx = max(maxx, x + w)
                maxy = max(maxy, y + h)
            x, y, w, h = minx, miny, maxx - minx, maxy - miny
            cropped = image[y:min(img_h, y+h+NUM_PX_COMMA), x:min(img_w, x+w)]
        else:
            # If we morphed out all of the text, assume an empty image.
            cropped = MAX_COLOR_VAL * np.ones(shape=(20, 100), dtype=np.uint8)
        bordered = cv2.copyMakeBorder(cropped, 5, 5, 5, 5, cv2.BORDER_CONSTANT, None, 255)
        return bordered
    def ocr_image(image, config):
        return pytesseract.image_to_string(
            image,
            lang='eng', config='--psm 6'
        )

    file_path = os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input',
                                os.path.splitext(file_name)[0],'cells')
    image_file = os.listdir(file_path)
    tess_args = os.listdir(file_path)

    main(image_file, tess_args)
    
    def text_files_to_csv(files):
        """Files must be sorted lexicographically
        Filenames must be <row>-<colum>.txt.
        000-000.txt
        000-001.txt
        001-000.txt
        etc...
        """
        rows = []
        for f in files:
            directory, filename = os.path.split(f)
            with open(os.path.join(file_path,f)) as of:
                txt = of.read().strip()
            row, column = map(int, filename.split(".")[0].split("-"))
            if row == len(rows):
                rows.append([])
            rows[row].append(txt)

        csv_file = io.StringIO()
        writer = csv.writer(csv_file)
        writer.writerows(rows)
        return csv_file.getvalue()

    def main(files):
        return text_files_to_csv(files)


    file_path = os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input',
                                os.path.splitext(file_name)[0],'text')
    file = os.listdir(file_path)
    files = sorted(file)

    a = main(files)
    #print(a)

    c = os.path.splitext(file_name)[0]+'_text.txt'
    t = os.path.splitext(file_name)[0]+'_f-text.txt'

    #cs = os.path.splitext(file_name)[0]+'_xl.csv'

    text_file = open(os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input',
                                os.path.splitext(file_name)[0],'txt',c), "wt")
    n = text_file.write(a)
    text_file.close()

    df = pd.read_csv(os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input',
                                os.path.splitext(file_name)[0],'txt',c),
        header=None,delimiter=',',names=list(range(10)))
    #print(df)
    
    df1 = df.dropna(axis = 0,how = 'all')
    df1[1] = df1[1].astype(str)

    #print(df1)
    
    word = 'TOTAL WAGES PAID'
    
        
    
    if df_drop[0].dtypes != 'object':
        print(os.path.splitext(file_name)[0]+'.tif','TOTAL WAGES PAID',0)
        
    else:
        
        for i in df1[0]:
            if fuzz.ratio(word,i) > 80:
                #print(i)
                dff = df1[df1[0] == i]
                dff = dff.dropna(axis = 1,how = 'all')
                output = dff[1]
                print(output)
                output = re.sub("[ ,.#*!@$%^&()|\''"":;]", "", output)
                #print(output)
                if len(output) == 0:
                    print(os.path.splitext(file_name)[0]+'.tif','TOTAL WAGES PAID',0)
                else:
                    with open(os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input',
                                            os.path.splitext(file_name)[0],'final_text',t), 'a') as f:
                        f.write(output.to_string(header = False, index = False))
                    text_file = open(os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input',
                                            os.path.splitext(file_name)[0],'final_text',t), 'r+')
                    print(os.path.splitext(file_name)[0]+'.tif','TOTAL WAGES PAID',text_file.read())

2    907.54
Name: 1, dtype: object


TypeError: expected string or bytes-like object

In [53]:
df

,0,1,2,3,4,5,6,7,8,9
0,2 MO EMPLOYER ACCOUNT NO YEAR\n\nae : 5 Ah 202...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"MUST HAVE AMOUNTS IN 4, 5, & 6, EVEN IF ZERO",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4 TOTAL WAGES PAID,907.54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5 WAGES PAID IN EXCESS OF\nPER WORKER\nPER YEA...,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [54]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       4 non-null      object 
 1   1       2 non-null      float64
 2   2       0 non-null      float64
 3   3       0 non-null      float64
 4   4       0 non-null      float64
 5   5       0 non-null      float64
 6   6       0 non-null      float64
 7   7       0 non-null      float64
 8   8       0 non-null      float64
 9   9       0 non-null      float64
dtypes: float64(9), object(1)
memory usage: 448.0+ bytes


In [55]:
df1

,0,1,2,3,4,5,6,7,8,9
0,2 MO EMPLOYER ACCOUNT NO YEAR\n\nae : 5 Ah 202...,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"MUST HAVE AMOUNTS IN 4, 5, & 6, EVEN IF ZERO",nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4 TOTAL WAGES PAID,907.54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5 WAGES PAID IN EXCESS OF\nPER WORKER\nPER YEA...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [56]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4 entries, 0 to 3
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       4 non-null      object 
 1   1       4 non-null      object 
 2   2       0 non-null      float64
 3   3       0 non-null      float64
 4   4       0 non-null      float64
 5   5       0 non-null      float64
 6   6       0 non-null      float64
 7   7       0 non-null      float64
 8   8       0 non-null      float64
 9   9       0 non-null      float64
dtypes: float64(8), object(2)
memory usage: 352.0+ bytes


In [50]:
df[1] = df[1].astype(str)

In [17]:
df1 = df.dropna(axis = 0,how = 'all')
df1

,0,1,2,3,4,5,6,7,8,9
0,2 MO EMPLOYER ACCOUNT NO YEAR\n\nae : 5 Ah 202...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"MUST HAVE AMOUNTS IN 4, 5, & 6, EVEN IF ZERO",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4 TOTAL WAGES PAID,907.54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5 WAGES PAID IN EXCESS OF\nPER WORKER\nPER YEA...,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
for i in df1[0]:
            if fuzz.ratio(word,i) > 80:
                #print(i)
                dff = df1[df1[0] == i]
                dff = dff.dropna(axis = 1,how = 'all')
                output = (dff[1])
                print(output)

2    907.54
Name: 1, dtype: float64


In [39]:
output

0    907.54
Name: 1, dtype: float64

In [37]:
out = output.reset_index(drop=True, inplace=True)
out

In [31]:
output = output.style.hide_index()
output

AttributeError: 'Series' object has no attribute 'style'

In [16]:
import re
num_1 = re.sub("[ ,.#*!@$%^&()|\''"":;]", "", output)
num_1

'290754\nName1dtypefloat64'

In [23]:
df1

,0,1,2,3,4,5,6,7,8,9
0,2 MO EMPLOYER ACCOUNT NO YEAR\n\nae : 5 Ah 202...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"MUST HAVE AMOUNTS IN 4, 5, & 6, EVEN IF ZERO",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4 TOTAL WAGES PAID,907.54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5 WAGES PAID IN EXCESS OF\nPER WORKER\nPER YEA...,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
df2 = df1.style.hide_index()
df2

0,1,2,3,4,5,6,7,8,9
2 MO EMPLOYER ACCOUNT NO YEAR ae : 5 Ah 2020 3 CALENDAR QUARTER it [x] 2 [} at FO] am (J,nan,nan,nan,nan,nan,nan,nan,nan,nan
"MUST HAVE AMOUNTS IN 4, 5, & 6, EVEN IF ZERO",nan,nan,nan,nan,nan,nan,nan,nan,nan
4 TOTAL WAGES PAID,907.540000,nan,nan,nan,nan,nan,nan,nan,nan
5 WAGES PAID IN EXCESS OF PER WORKER PER YEAR (See Instruction Sheet),0.000000,nan,nan,nan,nan,nan,nan,nan,nan


In [70]:
    import matplotlib.image as mpimg
    import matplotlib.pyplot as plt
    from PIL import Image
    import cv2
    import os
    import math
    import sys
    import numpy as np
    import pytesseract
    import io 
    import csv
    import pandas as pd
    import shutil, os
    import warnings
    warnings.filterwarnings('ignore')
    
    file_name = '0500847001.jpeg'

    #path = '/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input/input'
    #for file_name in os.listdir(path):
        #print(file_name)
    shutil.copytree('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input/sample', 
        os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input',
                                 os.path.splitext(file_name)[0]))
     
    img = mpimg.imread(os.path.join
                       ('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input/input',
                        file_name))
    width = img.shape[1]
    height = img.shape[0]
    wi = width * 0.95
    he = height * 0.3
    w = int(wi)
    h = int(he)
    y = 0
    x1 = w * 0.45
    x = int(x1)
    crop = img[y:y+h, x:x+w]
    #plt.imshow(crop)
    im = Image.fromarray(crop)
    b = os.path.splitext(file_name)[0]+'_crop.jpeg'
    im.save(os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input',
                           os.path.splitext(file_name)[0],'crop',b))
    def extract_cell_images_from_table(image):
        BLUR_KERNEL_SIZE = (17, 17)
        STD_DEV_X_DIRECTION = 0
        STD_DEV_Y_DIRECTION = 0
        blurred = cv2.GaussianBlur(image, BLUR_KERNEL_SIZE, STD_DEV_X_DIRECTION, STD_DEV_Y_DIRECTION)
        MAX_COLOR_VAL = 255
        BLOCK_SIZE = 15
        SUBTRACT_FROM_MEAN = -2

        img_bin = cv2.adaptiveThreshold(
            ~blurred,
            MAX_COLOR_VAL,
            cv2.ADAPTIVE_THRESH_MEAN_C,
            cv2.THRESH_BINARY,
            BLOCK_SIZE,
            SUBTRACT_FROM_MEAN,
        )
        vertical = horizontal = img_bin.copy()
        SCALE = 5
        image_width, image_height = horizontal.shape
        horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (int(image_width / SCALE), 1))
        horizontally_opened = cv2.morphologyEx(img_bin, cv2.MORPH_OPEN, horizontal_kernel)
        vertical_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, int(image_height / SCALE)))
        vertically_opened = cv2.morphologyEx(img_bin, cv2.MORPH_OPEN, vertical_kernel)

        horizontally_dilated = cv2.dilate(horizontally_opened, cv2.getStructuringElement(cv2.MORPH_RECT, (40, 1)))
        vertically_dilated = cv2.dilate(vertically_opened, cv2.getStructuringElement(cv2.MORPH_RECT, (1, 60)))

        mask = horizontally_dilated + vertically_dilated
        contours, heirarchy = cv2.findContours(
            mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE,
        )

        perimeter_lengths = [cv2.arcLength(c, True) for c in contours]
        epsilons = [0.05 * p for p in perimeter_lengths]
        approx_polys = [cv2.approxPolyDP(c, e, True) for c, e in zip(contours, epsilons)]

        # Filter out contours that aren't rectangular. Those that aren't rectangular
        # are probably noise.
        approx_rects = [p for p in approx_polys if len(p) == 4]
        bounding_rects = [cv2.boundingRect(a) for a in approx_polys]

        # Filter out rectangles that are too narrow or too short.
        MIN_RECT_WIDTH = 40
        MIN_RECT_HEIGHT = 10
        bounding_rects = [
            r for r in bounding_rects if MIN_RECT_WIDTH < r[2] and MIN_RECT_HEIGHT < r[3]
        ]

        # The largest bounding rectangle is assumed to be the entire table.
        # Remove it from the list. We don't want to accidentally try to OCR
        # the entire table.
        largest_rect = max(bounding_rects, key=lambda r: r[2] * r[3])
        bounding_rects = [b for b in bounding_rects if b is not largest_rect]

        cells = [c for c in bounding_rects]
        def cell_in_same_row(c1, c2):
            c1_center = c1[1] + c1[3] - c1[3] / 2
            c2_bottom = c2[1] + c2[3]
            c2_top = c2[1]
            return c2_top < c1_center < c2_bottom

        orig_cells = [c for c in cells]
        rows = []
        while cells:
            first = cells[0]
            rest = cells[1:]
            cells_in_same_row = sorted(
                [
                    c for c in rest
                    if cell_in_same_row(c, first)
                ],
                key=lambda c: c[0]
            )

            row_cells = sorted([first] + cells_in_same_row, key=lambda c: c[0])
            rows.append(row_cells)
            cells = [
                c for c in rest
                if not cell_in_same_row(c, first)
            ]

        # Sort rows by average height of their center.
        def avg_height_of_center(row):
            centers = [y + h - h / 2 for x, y, w, h in row]
            return sum(centers) / len(centers)

        rows.sort(key=avg_height_of_center)
        cell_images_rows = []
        for row in rows:
            cell_images_row = []
            for x, y, w, h in row:
                cell_images_row.append(image[y:y+h, x:x+w])
            cell_images_rows.append(cell_images_row)
        return cell_images_rows

    def main(f):
        results = []
        directory, filename = os.path.split(f)
        table = cv2.imread(f, cv2.IMREAD_GRAYSCALE)
        rows = extract_cell_images_from_table(table)
        #cell_img_dir = os.path.join(directory, "cells")
        #os.makedirs(cell_img_dir, exist_ok=True)
        out_path = os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input',
                                os.path.splitext(file_name)[0],'cells')
        paths = []
        for i, row in enumerate(rows):
            for j, cell in enumerate(row):
                cell_filename = "{:03d}-{:03d}.png".format(i, j)
                path = os.path.join(out_path, cell_filename)
                cv2.imwrite(path, cell)
                paths.append(path)
        return paths

    f = os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input',
                           os.path.splitext(file_name)[0],'crop',b)
    main(f)
    
    def main(image_file, tess_args):
        """
        OCR the image and output the text to a file with an extension that is ready
        to be used in Tesseract training (.gt.txt).
        Tries to crop the image so that only the relevant text gets passed to Tesseract.
        Returns the name of the text file that contains the text.
        """
        #file_path = '/home/vimal/Documents/table_detect_samples/structured images/input/011364700/cells'
        #image_file = os.listdir(file_path)

        for f in image_file:
            #print(f)
            directory, filename = os.path.split(f)
            filename_sans_ext, ext = os.path.splitext(filename)
            image = cv2.imread(os.path.join(file_path,f), cv2.IMREAD_GRAYSCALE)
            cropped = crop_to_text(image)
            #ocr_data_dir = os.path.join(directory, "ocr_data")
            #os.makedirs(ocr_data_dir, exist_ok=True)
            #out_imagepath = os.path.join(ocr_data_dir, filename)
            out_path = os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input',
                                os.path.splitext(file_name)[0],'text')
            out_txtpath = os.path.join(out_path, "{}.txt".format(filename_sans_ext))
            #cv2.imwrite(out_imagepath, cropped)
            if not tess_args:
                d = os.path.dirname(sys.modules["table_ocr"].__file__)
                tessdata_dir = os.path.join(d, "tessdata")
                tess_args = ["--psm", "7", "-l", "table-ocr", "--tessdata-dir", tessdata_dir]
            txt = ocr_image(cropped, " ".join(tess_args))
            with open(out_txtpath, "w") as txt_file:
                txt_file.write(txt)
            #return out_txtpath
    def crop_to_text(image):
        MAX_COLOR_VAL = 255
        BLOCK_SIZE = 15
        SUBTRACT_FROM_MEAN = -2

        img_bin = cv2.adaptiveThreshold(
            ~image,
            MAX_COLOR_VAL,
            cv2.ADAPTIVE_THRESH_MEAN_C,
            cv2.THRESH_BINARY,
            BLOCK_SIZE,
            SUBTRACT_FROM_MEAN,
        )

        img_h, img_w = image.shape
        horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (int(img_w * 0.5), 1))
        vertical_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, int(img_h * 0.7)))
        horizontal_lines = cv2.morphologyEx(img_bin, cv2.MORPH_OPEN, horizontal_kernel)
        vertical_lines = cv2.morphologyEx(img_bin, cv2.MORPH_OPEN, vertical_kernel)
        both = horizontal_lines + vertical_lines
        cleaned = img_bin - both

        # Get rid of little noise.
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
        opened = cv2.morphologyEx(cleaned, cv2.MORPH_OPEN, kernel)
        opened = cv2.dilate(opened, kernel)

        contours, hierarchy = cv2.findContours(opened, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
        bounding_rects = [cv2.boundingRect(c) for c in contours]
        NUM_PX_COMMA = 6
        MIN_CHAR_AREA = 5 * 9
        char_sized_bounding_rects = [(x, y, w, h) for x, y, w, h in bounding_rects if w * h > MIN_CHAR_AREA]
        if char_sized_bounding_rects:
            minx, miny, maxx, maxy = math.inf, math.inf, 0, 0
            for x, y, w, h in char_sized_bounding_rects:
                minx = min(minx, x)
                miny = min(miny, y)
                maxx = max(maxx, x + w)
                maxy = max(maxy, y + h)
            x, y, w, h = minx, miny, maxx - minx, maxy - miny
            cropped = image[y:min(img_h, y+h+NUM_PX_COMMA), x:min(img_w, x+w)]
        else:
            # If we morphed out all of the text, assume an empty image.
            cropped = MAX_COLOR_VAL * np.ones(shape=(20, 100), dtype=np.uint8)
        bordered = cv2.copyMakeBorder(cropped, 5, 5, 5, 5, cv2.BORDER_CONSTANT, None, 255)
        return bordered
    def ocr_image(image, config):
        return pytesseract.image_to_string(
            image,
            lang='eng', config='--psm 6'
        )

    file_path = os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input',
                                os.path.splitext(file_name)[0],'cells')
    image_file = os.listdir(file_path)
    tess_args = os.listdir(file_path)

    main(image_file, tess_args)
    
    def text_files_to_csv(files):
        """Files must be sorted lexicographically
        Filenames must be <row>-<colum>.txt.
        000-000.txt
        000-001.txt
        001-000.txt
        etc...
        """
        rows = []
        for f in files:
            directory, filename = os.path.split(f)
            with open(os.path.join(file_path,f)) as of:
                txt = of.read().strip()
            row, column = map(int, filename.split(".")[0].split("-"))
            if row == len(rows):
                rows.append([])
            rows[row].append(txt)

        csv_file = io.StringIO()
        writer = csv.writer(csv_file)
        writer.writerows(rows)
        return csv_file.getvalue()

    def main(files):
        return text_files_to_csv(files)


    file_path = os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input',
                                os.path.splitext(file_name)[0],'text')
    file = os.listdir(file_path)
    files = sorted(file)

    a = main(files)
    #print(a)

    c = os.path.splitext(file_name)[0]+'_text.txt'
    t = os.path.splitext(file_name)[0]+'_f-text.txt'

    #cs = os.path.splitext(file_name)[0]+'_xl.csv'

    text_file = open(os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input',
                                os.path.splitext(file_name)[0],'txt',c), "wt")
    n = text_file.write(a)
    text_file.close()

    df = pd.read_csv(os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input',
                                os.path.splitext(file_name)[0],'txt',c),
        header=None,delimiter=',',names=list(range(10)))
    #print(df)
    
    df1 = df.dropna(axis = 0,how = 'all')
    #print(df1)
    
    word = 'TOTAL WAGES PAID'
    
        
    
    if df1[0].dtypes != 'object':
        print(os.path.splitext(file_name)[0]+'.tif','TOTAL WAGES PAID',0)
        
    else:
        
        for i in df1[0]:
            if fuzz.ratio(word,i) > 80:
                #print(i)
                dff = df1[df1[0] == i]
                dff = dff.dropna(axis = 1,how = 'all')
                print(dff)
                output = dff[1]
                print(output)
                out = re.sub("[ ,.#*!@$%^&()|\''"":;]", "", output)
                print(out)
                
                if len(output) == 0:
                    print(os.path.splitext(file_name)[0]+'.tif','TOTAL WAGES PAID',0)
                else:
                    with open(os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input',
                                            os.path.splitext(file_name)[0],'final_text',t), 'a') as f:
                        f.write(output.to_string(header = False, index = False))
                        
                    text_file = open(os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input',
                                            os.path.splitext(file_name)[0],'final_text',t), 'r+')
                    
                    print(os.path.splitext(file_name)[0]+'.tif','TOTAL WAGES PAID',text_file.read())
        
                
                
    
            
        

                    0       1
2  4 TOTAL WAGES PAID  907.54
2    907.54
Name: 1, dtype: float64


TypeError: expected string or bytes-like object

In [60]:
df

,0,1,2,3,4,5,6,7,8,9
0,2 MO EMPLOYER ACCOUNT NO YEAR\n\nae : 5 Ah 202...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"MUST HAVE AMOUNTS IN 4, 5, & 6, EVEN IF ZERO",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4 TOTAL WAGES PAID,907.54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5 WAGES PAID IN EXCESS OF\nPER WORKER\nPER YEA...,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [62]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       4 non-null      object 
 1   1       2 non-null      float64
 2   2       0 non-null      float64
 3   3       0 non-null      float64
 4   4       0 non-null      float64
 5   5       0 non-null      float64
 6   6       0 non-null      float64
 7   7       0 non-null      float64
 8   8       0 non-null      float64
 9   9       0 non-null      float64
dtypes: float64(9), object(1)
memory usage: 448.0+ bytes


In [63]:
df1

,0,1,2,3,4,5,6,7,8,9
0,2 MO EMPLOYER ACCOUNT NO YEAR\n\nae : 5 Ah 202...,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"MUST HAVE AMOUNTS IN 4, 5, & 6, EVEN IF ZERO",nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4 TOTAL WAGES PAID,907.54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5 WAGES PAID IN EXCESS OF\nPER WORKER\nPER YEA...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
def twp(file_name):
    import matplotlib.image as mpimg
    import matplotlib.pyplot as plt
    from PIL import Image
    from fuzzywuzzy import fuzz
    from fuzzywuzzy import process
    import cv2
    import os
    import math
    import sys
    import numpy as np
    import pytesseract
    import io 
    import csv
    import pandas as pd
    import shutil, os
    import warnings
    warnings.filterwarnings('ignore')
    
    #file_name = '0100477000.jpeg'

    #path = '/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input/input'
    #for file_name in os.listdir(path):
        #print(file_name)
    shutil.copytree('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input/sample', 
        os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input',
                                 os.path.splitext(file_name)[0]))
     
    img = mpimg.imread(os.path.join
                       ('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input/input',
                        file_name))
    width = img.shape[1]
    height = img.shape[0]
    wi = width * 0.95
    he = height * 0.3
    w = int(wi)
    h = int(he)
    y = 0
    x1 = w * 0.45
    x = int(x1)
    crop = img[y:y+h, x:x+w]
    #plt.imshow(crop)
    im = Image.fromarray(crop)
    b = os.path.splitext(file_name)[0]+'_crop.jpeg'
    im.save(os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input',
                           os.path.splitext(file_name)[0],'crop',b))
    def extract_cell_images_from_table(image):
        BLUR_KERNEL_SIZE = (17, 17)
        STD_DEV_X_DIRECTION = 0
        STD_DEV_Y_DIRECTION = 0
        blurred = cv2.GaussianBlur(image, BLUR_KERNEL_SIZE, STD_DEV_X_DIRECTION, STD_DEV_Y_DIRECTION)
        MAX_COLOR_VAL = 255
        BLOCK_SIZE = 15
        SUBTRACT_FROM_MEAN = -2

        img_bin = cv2.adaptiveThreshold(
            ~blurred,
            MAX_COLOR_VAL,
            cv2.ADAPTIVE_THRESH_MEAN_C,
            cv2.THRESH_BINARY,
            BLOCK_SIZE,
            SUBTRACT_FROM_MEAN,
        )
        vertical = horizontal = img_bin.copy()
        SCALE = 5
        image_width, image_height = horizontal.shape
        horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (int(image_width / SCALE), 1))
        horizontally_opened = cv2.morphologyEx(img_bin, cv2.MORPH_OPEN, horizontal_kernel)
        vertical_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, int(image_height / SCALE)))
        vertically_opened = cv2.morphologyEx(img_bin, cv2.MORPH_OPEN, vertical_kernel)

        horizontally_dilated = cv2.dilate(horizontally_opened, cv2.getStructuringElement(cv2.MORPH_RECT, (40, 1)))
        vertically_dilated = cv2.dilate(vertically_opened, cv2.getStructuringElement(cv2.MORPH_RECT, (1, 60)))

        mask = horizontally_dilated + vertically_dilated
        contours, heirarchy = cv2.findContours(
            mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE,
        )

        perimeter_lengths = [cv2.arcLength(c, True) for c in contours]
        epsilons = [0.05 * p for p in perimeter_lengths]
        approx_polys = [cv2.approxPolyDP(c, e, True) for c, e in zip(contours, epsilons)]

        # Filter out contours that aren't rectangular. Those that aren't rectangular
        # are probably noise.
        approx_rects = [p for p in approx_polys if len(p) == 4]
        bounding_rects = [cv2.boundingRect(a) for a in approx_polys]

        # Filter out rectangles that are too narrow or too short.
        MIN_RECT_WIDTH = 40
        MIN_RECT_HEIGHT = 10
        bounding_rects = [
            r for r in bounding_rects if MIN_RECT_WIDTH < r[2] and MIN_RECT_HEIGHT < r[3]
        ]

        # The largest bounding rectangle is assumed to be the entire table.
        # Remove it from the list. We don't want to accidentally try to OCR
        # the entire table.
        largest_rect = max(bounding_rects, key=lambda r: r[2] * r[3])
        bounding_rects = [b for b in bounding_rects if b is not largest_rect]

        cells = [c for c in bounding_rects]
        def cell_in_same_row(c1, c2):
            c1_center = c1[1] + c1[3] - c1[3] / 2
            c2_bottom = c2[1] + c2[3]
            c2_top = c2[1]
            return c2_top < c1_center < c2_bottom

        orig_cells = [c for c in cells]
        rows = []
        while cells:
            first = cells[0]
            rest = cells[1:]
            cells_in_same_row = sorted(
                [
                    c for c in rest
                    if cell_in_same_row(c, first)
                ],
                key=lambda c: c[0]
            )

            row_cells = sorted([first] + cells_in_same_row, key=lambda c: c[0])
            rows.append(row_cells)
            cells = [
                c for c in rest
                if not cell_in_same_row(c, first)
            ]

        # Sort rows by average height of their center.
        def avg_height_of_center(row):
            centers = [y + h - h / 2 for x, y, w, h in row]
            return sum(centers) / len(centers)

        rows.sort(key=avg_height_of_center)
        cell_images_rows = []
        for row in rows:
            cell_images_row = []
            for x, y, w, h in row:
                cell_images_row.append(image[y:y+h, x:x+w])
            cell_images_rows.append(cell_images_row)
        return cell_images_rows

    def main(f):
        results = []
        directory, filename = os.path.split(f)
        table = cv2.imread(f, cv2.IMREAD_GRAYSCALE)
        rows = extract_cell_images_from_table(table)
        #cell_img_dir = os.path.join(directory, "cells")
        #os.makedirs(cell_img_dir, exist_ok=True)
        out_path = os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input',
                                os.path.splitext(file_name)[0],'cells')
        paths = []
        for i, row in enumerate(rows):
            for j, cell in enumerate(row):
                cell_filename = "{:03d}-{:03d}.png".format(i, j)
                path = os.path.join(out_path, cell_filename)
                cv2.imwrite(path, cell)
                paths.append(path)
        return paths

    f = os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input',
                           os.path.splitext(file_name)[0],'crop',b)
    main(f)
    
    def main(image_file, tess_args):
        """
        OCR the image and output the text to a file with an extension that is ready
        to be used in Tesseract training (.gt.txt).
        Tries to crop the image so that only the relevant text gets passed to Tesseract.
        Returns the name of the text file that contains the text.
        """
        #file_path = '/home/vimal/Documents/table_detect_samples/structured images/input/011364700/cells'
        #image_file = os.listdir(file_path)

        for f in image_file:
            #print(f)
            directory, filename = os.path.split(f)
            filename_sans_ext, ext = os.path.splitext(filename)
            image = cv2.imread(os.path.join(file_path,f), cv2.IMREAD_GRAYSCALE)
            cropped = crop_to_text(image)
            #ocr_data_dir = os.path.join(directory, "ocr_data")
            #os.makedirs(ocr_data_dir, exist_ok=True)
            #out_imagepath = os.path.join(ocr_data_dir, filename)
            out_path = os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input',
                                os.path.splitext(file_name)[0],'text')
            out_txtpath = os.path.join(out_path, "{}.txt".format(filename_sans_ext))
            #cv2.imwrite(out_imagepath, cropped)
            if not tess_args:
                d = os.path.dirname(sys.modules["table_ocr"].__file__)
                tessdata_dir = os.path.join(d, "tessdata")
                tess_args = ["--psm", "7", "-l", "table-ocr", "--tessdata-dir", tessdata_dir]
            txt = ocr_image(cropped, " ".join(tess_args))
            with open(out_txtpath, "w") as txt_file:
                txt_file.write(txt)
            #return out_txtpath
    def crop_to_text(image):
        MAX_COLOR_VAL = 255
        BLOCK_SIZE = 15
        SUBTRACT_FROM_MEAN = -2

        img_bin = cv2.adaptiveThreshold(
            ~image,
            MAX_COLOR_VAL,
            cv2.ADAPTIVE_THRESH_MEAN_C,
            cv2.THRESH_BINARY,
            BLOCK_SIZE,
            SUBTRACT_FROM_MEAN,
        )

        img_h, img_w = image.shape
        horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (int(img_w * 0.5), 1))
        vertical_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, int(img_h * 0.7)))
        horizontal_lines = cv2.morphologyEx(img_bin, cv2.MORPH_OPEN, horizontal_kernel)
        vertical_lines = cv2.morphologyEx(img_bin, cv2.MORPH_OPEN, vertical_kernel)
        both = horizontal_lines + vertical_lines
        cleaned = img_bin - both

        # Get rid of little noise.
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
        opened = cv2.morphologyEx(cleaned, cv2.MORPH_OPEN, kernel)
        opened = cv2.dilate(opened, kernel)

        contours, hierarchy = cv2.findContours(opened, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
        bounding_rects = [cv2.boundingRect(c) for c in contours]
        NUM_PX_COMMA = 6
        MIN_CHAR_AREA = 5 * 9
        char_sized_bounding_rects = [(x, y, w, h) for x, y, w, h in bounding_rects if w * h > MIN_CHAR_AREA]
        if char_sized_bounding_rects:
            minx, miny, maxx, maxy = math.inf, math.inf, 0, 0
            for x, y, w, h in char_sized_bounding_rects:
                minx = min(minx, x)
                miny = min(miny, y)
                maxx = max(maxx, x + w)
                maxy = max(maxy, y + h)
            x, y, w, h = minx, miny, maxx - minx, maxy - miny
            cropped = image[y:min(img_h, y+h+NUM_PX_COMMA), x:min(img_w, x+w)]
        else:
            # If we morphed out all of the text, assume an empty image.
            cropped = MAX_COLOR_VAL * np.ones(shape=(20, 100), dtype=np.uint8)
        bordered = cv2.copyMakeBorder(cropped, 5, 5, 5, 5, cv2.BORDER_CONSTANT, None, 255)
        return bordered
    def ocr_image(image, config):
        return pytesseract.image_to_string(
            image,
            lang='eng', config='--psm 6'
        )

    file_path = os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input',
                                os.path.splitext(file_name)[0],'cells')
    image_file = os.listdir(file_path)
    tess_args = os.listdir(file_path)

    main(image_file, tess_args)
    
    def text_files_to_csv(files):
        """Files must be sorted lexicographically
        Filenames must be <row>-<colum>.txt.
        000-000.txt
        000-001.txt
        001-000.txt
        etc...
        """
        rows = []
        for f in files:
            directory, filename = os.path.split(f)
            with open(os.path.join(file_path,f)) as of:
                txt = of.read().strip()
            row, column = map(int, filename.split(".")[0].split("-"))
            if row == len(rows):
                rows.append([])
            rows[row].append(txt)

        csv_file = io.StringIO()
        writer = csv.writer(csv_file)
        writer.writerows(rows)
        return csv_file.getvalue()

    def main(files):
        return text_files_to_csv(files)


    file_path = os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input',
                                os.path.splitext(file_name)[0],'text')
    file = os.listdir(file_path)
    files = sorted(file)

    a = main(files)
    #print(a)

    c = os.path.splitext(file_name)[0]+'_text.txt'
    t = os.path.splitext(file_name)[0]+'_f-text.txt'

    #cs = os.path.splitext(file_name)[0]+'_xl.csv'

    text_file = open(os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input',
                                os.path.splitext(file_name)[0],'txt',c), "wt")
    n = text_file.write(a)
    text_file.close()

    df = pd.read_csv(os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input',
                                os.path.splitext(file_name)[0],'txt',c),
        header=None,delimiter=',',names=list(range(10)))
    #print(df)
    
    df1 = df.dropna(axis = 0,how = 'all')
    #print(df1)
    
    word = 'TOTAL WAGES PAID'
    
        
    
    if df1[0].dtypes != 'object':
        print(os.path.splitext(file_name)[0]+'.tif','TOTAL WAGES PAID',0)
        
    else:
        
        for i in df1[0]:
            if fuzz.ratio(word,i) > 80:
                #print(i)
                dff = df1[df1[0] == i]
                dff = dff.dropna(axis = 1,how = 'all')
                output = dff[1]
                output = output.str.replace(r'[^\w\s]+', '')
                output = output.str.replace(" ","")
                #print(output)
                
                if len(output) == 0:
                    print(os.path.splitext(file_name)[0]+'.tif','TOTAL WAGES PAID',0)
                else:
                    with open(os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input',
                                            os.path.splitext(file_name)[0],'final_text',t), 'a') as f:
                        f.write(output.to_string(header = False, index = False))
                        
                    text_file = open(os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input',
                                            os.path.splitext(file_name)[0],'final_text',t), 'r+')
                    
                    print(os.path.splitext(file_name)[0]+'.tif','TOTAL WAGES PAID',text_file.read())
        
                
                
    
            
        

In [11]:
file_name = '0101231000.jpeg'
twp(file_name)

0101231000.tif TOTAL WAGES PAID 99454


In [1]:
def twgp()    
    import matplotlib.image as mpimg
    import matplotlib.pyplot as plt
    from PIL import Image
    from fuzzywuzzy import fuzz
    from fuzzywuzzy import process
    import cv2
    import os
    import math
    import sys
    import numpy as np
    import pytesseract
    import io 
    import csv
    import pandas as pd
    import shutil, os
    import warnings
    warnings.filterwarnings('ignore')
    
    #file_name = '0100477000.jpeg'

    path = '/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input/input'
    for file_name in os.listdir(path):
        #print(file_name)
        shutil.copytree('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input/sample', 
            os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input',
                                     os.path.splitext(file_name)[0]))

        img = mpimg.imread(os.path.join
                           ('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input/input',
                            file_name))
        width = img.shape[1]
        height = img.shape[0]
        wi = width * 0.95
        he = height * 0.3
        w = int(wi)
        h = int(he)
        y = 0
        x1 = w * 0.45
        x = int(x1)
        crop = img[y:y+h, x:x+w]
        #plt.imshow(crop)
        im = Image.fromarray(crop)
        b = os.path.splitext(file_name)[0]+'_crop.jpeg'
        im.save(os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input',
                               os.path.splitext(file_name)[0],'crop',b))
        def extract_cell_images_from_table(image):
            BLUR_KERNEL_SIZE = (17, 17)
            STD_DEV_X_DIRECTION = 0
            STD_DEV_Y_DIRECTION = 0
            blurred = cv2.GaussianBlur(image, BLUR_KERNEL_SIZE, STD_DEV_X_DIRECTION, STD_DEV_Y_DIRECTION)
            MAX_COLOR_VAL = 255
            BLOCK_SIZE = 15
            SUBTRACT_FROM_MEAN = -2

            img_bin = cv2.adaptiveThreshold(
                ~blurred,
                MAX_COLOR_VAL,
                cv2.ADAPTIVE_THRESH_MEAN_C,
                cv2.THRESH_BINARY,
                BLOCK_SIZE,
                SUBTRACT_FROM_MEAN,
            )
            vertical = horizontal = img_bin.copy()
            SCALE = 5
            image_width, image_height = horizontal.shape
            horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (int(image_width / SCALE), 1))
            horizontally_opened = cv2.morphologyEx(img_bin, cv2.MORPH_OPEN, horizontal_kernel)
            vertical_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, int(image_height / SCALE)))
            vertically_opened = cv2.morphologyEx(img_bin, cv2.MORPH_OPEN, vertical_kernel)

            horizontally_dilated = cv2.dilate(horizontally_opened, cv2.getStructuringElement(cv2.MORPH_RECT, (40, 1)))
            vertically_dilated = cv2.dilate(vertically_opened, cv2.getStructuringElement(cv2.MORPH_RECT, (1, 60)))

            mask = horizontally_dilated + vertically_dilated
            contours, heirarchy = cv2.findContours(
                mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE,
            )

            perimeter_lengths = [cv2.arcLength(c, True) for c in contours]
            epsilons = [0.05 * p for p in perimeter_lengths]
            approx_polys = [cv2.approxPolyDP(c, e, True) for c, e in zip(contours, epsilons)]

            # Filter out contours that aren't rectangular. Those that aren't rectangular
            # are probably noise.
            approx_rects = [p for p in approx_polys if len(p) == 4]
            bounding_rects = [cv2.boundingRect(a) for a in approx_polys]

            # Filter out rectangles that are too narrow or too short.
            MIN_RECT_WIDTH = 40
            MIN_RECT_HEIGHT = 10
            bounding_rects = [
                r for r in bounding_rects if MIN_RECT_WIDTH < r[2] and MIN_RECT_HEIGHT < r[3]
            ]

            # The largest bounding rectangle is assumed to be the entire table.
            # Remove it from the list. We don't want to accidentally try to OCR
            # the entire table.
            largest_rect = max(bounding_rects, key=lambda r: r[2] * r[3])
            bounding_rects = [b for b in bounding_rects if b is not largest_rect]

            cells = [c for c in bounding_rects]
            def cell_in_same_row(c1, c2):
                c1_center = c1[1] + c1[3] - c1[3] / 2
                c2_bottom = c2[1] + c2[3]
                c2_top = c2[1]
                return c2_top < c1_center < c2_bottom

            orig_cells = [c for c in cells]
            rows = []
            while cells:
                first = cells[0]
                rest = cells[1:]
                cells_in_same_row = sorted(
                    [
                        c for c in rest
                        if cell_in_same_row(c, first)
                    ],
                    key=lambda c: c[0]
                )

                row_cells = sorted([first] + cells_in_same_row, key=lambda c: c[0])
                rows.append(row_cells)
                cells = [
                    c for c in rest
                    if not cell_in_same_row(c, first)
                ]

            # Sort rows by average height of their center.
            def avg_height_of_center(row):
                centers = [y + h - h / 2 for x, y, w, h in row]
                return sum(centers) / len(centers)

            rows.sort(key=avg_height_of_center)
            cell_images_rows = []
            for row in rows:
                cell_images_row = []
                for x, y, w, h in row:
                    cell_images_row.append(image[y:y+h, x:x+w])
                cell_images_rows.append(cell_images_row)
            return cell_images_rows

        def main(f):
            results = []
            directory, filename = os.path.split(f)
            table = cv2.imread(f, cv2.IMREAD_GRAYSCALE)
            rows = extract_cell_images_from_table(table)
            #cell_img_dir = os.path.join(directory, "cells")
            #os.makedirs(cell_img_dir, exist_ok=True)
            out_path = os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input',
                                    os.path.splitext(file_name)[0],'cells')
            paths = []
            for i, row in enumerate(rows):
                for j, cell in enumerate(row):
                    cell_filename = "{:03d}-{:03d}.png".format(i, j)
                    path = os.path.join(out_path, cell_filename)
                    cv2.imwrite(path, cell)
                    paths.append(path)
            return paths

        f = os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input',
                               os.path.splitext(file_name)[0],'crop',b)
        main(f)

        def main(image_file, tess_args):
            """
            OCR the image and output the text to a file with an extension that is ready
            to be used in Tesseract training (.gt.txt).
            Tries to crop the image so that only the relevant text gets passed to Tesseract.
            Returns the name of the text file that contains the text.
            """
            #file_path = '/home/vimal/Documents/table_detect_samples/structured images/input/011364700/cells'
            #image_file = os.listdir(file_path)

            for f in image_file:
                #print(f)
                directory, filename = os.path.split(f)
                filename_sans_ext, ext = os.path.splitext(filename)
                image = cv2.imread(os.path.join(file_path,f), cv2.IMREAD_GRAYSCALE)
                cropped = crop_to_text(image)
                #ocr_data_dir = os.path.join(directory, "ocr_data")
                #os.makedirs(ocr_data_dir, exist_ok=True)
                #out_imagepath = os.path.join(ocr_data_dir, filename)
                out_path = os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input',
                                    os.path.splitext(file_name)[0],'text')
                out_txtpath = os.path.join(out_path, "{}.txt".format(filename_sans_ext))
                #cv2.imwrite(out_imagepath, cropped)
                if not tess_args:
                    d = os.path.dirname(sys.modules["table_ocr"].__file__)
                    tessdata_dir = os.path.join(d, "tessdata")
                    tess_args = ["--psm", "7", "-l", "table-ocr", "--tessdata-dir", tessdata_dir]
                txt = ocr_image(cropped, " ".join(tess_args))
                with open(out_txtpath, "w") as txt_file:
                    txt_file.write(txt)
                #return out_txtpath
        def crop_to_text(image):
            MAX_COLOR_VAL = 255
            BLOCK_SIZE = 15
            SUBTRACT_FROM_MEAN = -2

            img_bin = cv2.adaptiveThreshold(
                ~image,
                MAX_COLOR_VAL,
                cv2.ADAPTIVE_THRESH_MEAN_C,
                cv2.THRESH_BINARY,
                BLOCK_SIZE,
                SUBTRACT_FROM_MEAN,
            )

            img_h, img_w = image.shape
            horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (int(img_w * 0.5), 1))
            vertical_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, int(img_h * 0.7)))
            horizontal_lines = cv2.morphologyEx(img_bin, cv2.MORPH_OPEN, horizontal_kernel)
            vertical_lines = cv2.morphologyEx(img_bin, cv2.MORPH_OPEN, vertical_kernel)
            both = horizontal_lines + vertical_lines
            cleaned = img_bin - both

            # Get rid of little noise.
            kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
            opened = cv2.morphologyEx(cleaned, cv2.MORPH_OPEN, kernel)
            opened = cv2.dilate(opened, kernel)

            contours, hierarchy = cv2.findContours(opened, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
            bounding_rects = [cv2.boundingRect(c) for c in contours]
            NUM_PX_COMMA = 6
            MIN_CHAR_AREA = 5 * 9
            char_sized_bounding_rects = [(x, y, w, h) for x, y, w, h in bounding_rects if w * h > MIN_CHAR_AREA]
            if char_sized_bounding_rects:
                minx, miny, maxx, maxy = math.inf, math.inf, 0, 0
                for x, y, w, h in char_sized_bounding_rects:
                    minx = min(minx, x)
                    miny = min(miny, y)
                    maxx = max(maxx, x + w)
                    maxy = max(maxy, y + h)
                x, y, w, h = minx, miny, maxx - minx, maxy - miny
                cropped = image[y:min(img_h, y+h+NUM_PX_COMMA), x:min(img_w, x+w)]
            else:
                # If we morphed out all of the text, assume an empty image.
                cropped = MAX_COLOR_VAL * np.ones(shape=(20, 100), dtype=np.uint8)
            bordered = cv2.copyMakeBorder(cropped, 5, 5, 5, 5, cv2.BORDER_CONSTANT, None, 255)
            return bordered
        def ocr_image(image, config):
            return pytesseract.image_to_string(
                image,
                lang='eng', config='--psm 11'
            )

        file_path = os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input',
                                    os.path.splitext(file_name)[0],'cells')
        image_file = os.listdir(file_path)
        tess_args = os.listdir(file_path)

        main(image_file, tess_args)

        def text_files_to_csv(files):
            """Files must be sorted lexicographically
            Filenames must be <row>-<colum>.txt.
            000-000.txt
            000-001.txt
            001-000.txt
            etc...
            """
            rows = []
            for f in files:
                directory, filename = os.path.split(f)
                with open(os.path.join(file_path,f)) as of:
                    txt = of.read().strip()
                row, column = map(int, filename.split(".")[0].split("-"))
                if row == len(rows):
                    rows.append([])
                rows[row].append(txt)

            csv_file = io.StringIO()
            writer = csv.writer(csv_file)
            writer.writerows(rows)
            return csv_file.getvalue()

        def main(files):
            return text_files_to_csv(files)


        file_path = os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input',
                                    os.path.splitext(file_name)[0],'text')
        file = os.listdir(file_path)
        files = sorted(file)

        a = main(files)
        #print(a)

        c = os.path.splitext(file_name)[0]+'_text.txt'
        t = os.path.splitext(file_name)[0]+'_f-text.txt'

        #cs = os.path.splitext(file_name)[0]+'_xl.csv'

        text_file = open(os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input',
                                    os.path.splitext(file_name)[0],'txt',c), "wt")
        n = text_file.write(a)
        text_file.close()

        df = pd.read_csv(os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input',
                                    os.path.splitext(file_name)[0],'txt',c),
            header=None,delimiter=',',names=list(range(10)))
        df[1] = df[1].astype(str)
        #print(df)

        df1 = df.dropna(axis = 0,how = 'all')
        #print(df1)

        word = 'TOTAL WAGES PAID'



        if df1[0].dtypes != 'object':
            print(os.path.splitext(file_name)[0]+'.tif','TOTAL WAGES PAID',0)

        else:

            for i in df1[0]:
                if fuzz.ratio(word,str(i)) > 80:
                    #print(i)
                    dff = df1[df1[0] == i]
                    dff = dff.dropna(axis = 1,how = 'all')
                    output = dff[1]
                    output = output.str.replace(r'[^\w\s]+', '')
                    output = output.str.replace(" ","")
                    #print(output)

                    if len(output) == 0:
                        print(os.path.splitext(file_name)[0]+'.tif','TOTAL WAGES PAID',0)
                    else:
                        with open(os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input',
                                                os.path.splitext(file_name)[0],'final_text',t), 'a') as f:
                            f.write(output.to_string(header = False, index = False))

                        text_file = open(os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input',
                                                os.path.splitext(file_name)[0],'final_text',t), 'r+')

                        print(os.path.splitext(file_name)[0]+'.tif','TOTAL WAGES PAID',text_file.read())

/home/vimalkumar.s/Documents/python_new/cde_venv/lib/python3.8/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


0414782000.tif TOTAL WAGES PAID 10215000
0388041000.tif TOTAL WAGES PAID 1855002
0965510000.tif TOTAL WAGES PAID 39043
0964684000.tif TOTAL WAGES PAID 775000
0943362000.tif TOTAL WAGES PAID 4078454
0925469000.tif TOTAL WAGES PAID OLS4
0949784000.tif TOTAL WAGES PAID 325050
0774334000.tif TOTAL WAGES PAID 1823726
0843659000.tif TOTAL WAGES PAID 3103750
0689178000.tif TOTAL WAGES PAID 3922854
0216226000.tif TOTAL WAGES PAID 5650845
0648125000.tif TOTAL WAGES PAID 2090175
0822217000.tif TOTAL WAGES PAID 1013900
0343681000.tif TOTAL WAGES PAID 4026350
0876479000.tif TOTAL WAGES PAID nan
0415680002.tif TOTAL WAGES PAID 3475
0644730000.tif TOTAL WAGES PAID 2281796
0939535000.tif TOTAL WAGES PAID 34100C
0784699000.tif TOTAL WAGES PAID 285626
0796110000.tif TOTAL WAGES PAID 5
0469665000.tif TOTAL WAGES PAID 6688888
0539605000.tif TOTAL WAGES PAID 570000
0105253000.tif TOTAL WAGES PAID 1730296
0848314000.tif TOTAL WAGES PAID 364383
0507327000.tif TOTAL WAGES PAID 3175149
0906601000.tif TOTAL WA

0816899000.tif TOTAL WAGES PAID 4629
0857612000.tif TOTAL WAGES PAID 6416926
0603643000.tif TOTAL WAGES PAID 2507305
0792638000.tif TOTAL WAGES PAID 1032000
0414134001.tif TOTAL WAGES PAID 419100
0173923000.tif TOTAL WAGES PAID 2970849
0746730000.tif TOTAL WAGES PAID 21532143
0798409000.tif TOTAL WAGES PAID 864316
0941240000.tif TOTAL WAGES PAID 2927500
0745433000.tif TOTAL WAGES PAID 5255322
0889626000.tif TOTAL WAGES PAID 1249998
0922941001.tif TOTAL WAGES PAID 209894
0557316000.tif TOTAL WAGES PAID 600
0868133001.tif TOTAL WAGES PAID 334946
0849127000.tif TOTAL WAGES PAID 12497088
0716321001.tif TOTAL WAGES PAID 338675
0954438000.tif TOTAL WAGES PAID 3556047
0423173000.tif TOTAL WAGES PAID 4963118
0137136000.tif TOTAL WAGES PAID 7299257
0865388000.tif TOTAL WAGES PAID 00500
0892831000.tif TOTAL WAGES PAID 3908800
0830415000.tif TOTAL WAGES PAID 44799
0598048000.tif TOTAL WAGES PAID 3970751
0560522000.tif TOTAL WAGES PAID 28359226
0872485000.tif TOTAL WAGES PAID 8414515
0484371000.ti

0450140000.tif TOTAL WAGES PAID 446670
0737250000.tif TOTAL WAGES PAID 14818243
0621386000.tif TOTAL WAGES PAID 2053848
0118105000.tif TOTAL WAGES PAID 4672925
0450154000.tif TOTAL WAGES PAID 86S494
0852526000.tif TOTAL WAGES PAID 940652
0633979000.tif TOTAL WAGES PAID 3595576
0381002000.tif TOTAL WAGES PAID 627100
0904175000.tif TOTAL WAGES PAID 289680
0305040000.tif TOTAL WAGES PAID 6050
0693359000.tif TOTAL WAGES PAID 6907166
0286199000.tif TOTAL WAGES PAID 785268
0535100000.tif TOTAL WAGES PAID 2145000
0623205000.tif TOTAL WAGES PAID 164310
0576298000.tif TOTAL WAGES PAID 239855
0906485000.tif TOTAL WAGES PAID 3685580
0681766000.tif TOTAL WAGES PAID 3229618
0885263000.tif TOTAL WAGES PAID 201025
0296923000.tif TOTAL WAGES PAID 79486
0734805000.tif TOTAL WAGES PAID 1001455
0801417000.tif TOTAL WAGES PAID 1138197
0587994000.tif TOTAL WAGES PAID 839504
0906474000.tif TOTAL WAGES PAID nan
0598493000.tif TOTAL WAGES PAID 5598650
0950476000.tif TOTAL WAGES PAID 327000
0365338000.tif TOTA

In [1]:
def twp(file_name):
    import matplotlib.image as mpimg
    import matplotlib.pyplot as plt
    from PIL import Image
    from fuzzywuzzy import fuzz
    from fuzzywuzzy import process
    import cv2
    import os
    import math
    import sys
    import numpy as np
    import pytesseract
    import io 
    import csv
    import pandas as pd
    import shutil, os
    import warnings
    warnings.filterwarnings('ignore')
    
    #file_name = '0100477000.jpeg'

    #path = '/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input/input'
    #for file_name in os.listdir(path):
        #print(file_name)
    shutil.copytree('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input/sample', 
        os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input',
                                     os.path.splitext(file_name)[0]))

    img = mpimg.imread(os.path.join
                           ('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input/input',
                            file_name))
    width = img.shape[1]
    height = img.shape[0]
    wi = width * 0.95
    he = height * 0.3
    w = int(wi)
    h = int(he)
    y = 0
    x1 = w * 0.45
    x = int(x1)
    crop = img[y:y+h, x:x+w]
    #plt.imshow(crop)
    im = Image.fromarray(crop)
    b = os.path.splitext(file_name)[0]+'_crop.jpeg'
    im.save(os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input',
                               os.path.splitext(file_name)[0],'crop',b))
    def extract_cell_images_from_table(image):
        BLUR_KERNEL_SIZE = (17, 17)
        STD_DEV_X_DIRECTION = 0
        STD_DEV_Y_DIRECTION = 0
        blurred = cv2.GaussianBlur(image, BLUR_KERNEL_SIZE, STD_DEV_X_DIRECTION, STD_DEV_Y_DIRECTION)
        MAX_COLOR_VAL = 255
        BLOCK_SIZE = 15
        SUBTRACT_FROM_MEAN = -2

        img_bin = cv2.adaptiveThreshold(
            ~blurred,
            MAX_COLOR_VAL,
            cv2.ADAPTIVE_THRESH_MEAN_C,
            cv2.THRESH_BINARY,
            BLOCK_SIZE,
            SUBTRACT_FROM_MEAN,
        )
        vertical = horizontal = img_bin.copy()
        SCALE = 5
        image_width, image_height = horizontal.shape
        horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (int(image_width / SCALE), 1))
        horizontally_opened = cv2.morphologyEx(img_bin, cv2.MORPH_OPEN, horizontal_kernel)
        vertical_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, int(image_height / SCALE)))
        vertically_opened = cv2.morphologyEx(img_bin, cv2.MORPH_OPEN, vertical_kernel)

        horizontally_dilated = cv2.dilate(horizontally_opened, cv2.getStructuringElement(cv2.MORPH_RECT, (40, 1)))
        vertically_dilated = cv2.dilate(vertically_opened, cv2.getStructuringElement(cv2.MORPH_RECT, (1, 60)))

        mask = horizontally_dilated + vertically_dilated
        contours, heirarchy = cv2.findContours(
            mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE,
        )

        perimeter_lengths = [cv2.arcLength(c, True) for c in contours]
        epsilons = [0.05 * p for p in perimeter_lengths]
        approx_polys = [cv2.approxPolyDP(c, e, True) for c, e in zip(contours, epsilons)]

        # Filter out contours that aren't rectangular. Those that aren't rectangular
        # are probably noise.
        approx_rects = [p for p in approx_polys if len(p) == 4]
        bounding_rects = [cv2.boundingRect(a) for a in approx_polys]

        # Filter out rectangles that are too narrow or too short.
        MIN_RECT_WIDTH = 40
        MIN_RECT_HEIGHT = 10
        bounding_rects = [
            r for r in bounding_rects if MIN_RECT_WIDTH < r[2] and MIN_RECT_HEIGHT < r[3]
        ]

        # The largest bounding rectangle is assumed to be the entire table.
        # Remove it from the list. We don't want to accidentally try to OCR
        # the entire table.
        largest_rect = max(bounding_rects, key=lambda r: r[2] * r[3])
        bounding_rects = [b for b in bounding_rects if b is not largest_rect]

        cells = [c for c in bounding_rects]
        def cell_in_same_row(c1, c2):
            c1_center = c1[1] + c1[3] - c1[3] / 2
            c2_bottom = c2[1] + c2[3]
            c2_top = c2[1]
            return c2_top < c1_center < c2_bottom

        orig_cells = [c for c in cells]
        rows = []
        while cells:
            first = cells[0]
            rest = cells[1:]
            cells_in_same_row = sorted(
                [
                    c for c in rest
                    if cell_in_same_row(c, first)
                ],
                key=lambda c: c[0]
            )

            row_cells = sorted([first] + cells_in_same_row, key=lambda c: c[0])
            rows.append(row_cells)
            cells = [
                c for c in rest
                if not cell_in_same_row(c, first)
            ]

        # Sort rows by average height of their center.
        def avg_height_of_center(row):
            centers = [y + h - h / 2 for x, y, w, h in row]
            return sum(centers) / len(centers)

        rows.sort(key=avg_height_of_center)
        cell_images_rows = []
        for row in rows:
            cell_images_row = []
            for x, y, w, h in row:
                cell_images_row.append(image[y:y+h, x:x+w])
            cell_images_rows.append(cell_images_row)
        return cell_images_rows

    def main(f):
        results = []
        directory, filename = os.path.split(f)
        table = cv2.imread(f, cv2.IMREAD_GRAYSCALE)
        rows = extract_cell_images_from_table(table)
        #cell_img_dir = os.path.join(directory, "cells")
        #os.makedirs(cell_img_dir, exist_ok=True)
        out_path = os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input',
                                    os.path.splitext(file_name)[0],'cells')
        paths = []
        for i, row in enumerate(rows):
            for j, cell in enumerate(row):
                cell_filename = "{:03d}-{:03d}.png".format(i, j)
                path = os.path.join(out_path, cell_filename)
                cv2.imwrite(path, cell)
                paths.append(path)
        return paths

    f = os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input',
                               os.path.splitext(file_name)[0],'crop',b)
    main(f)

    def main(image_file, tess_args):
        """
        OCR the image and output the text to a file with an extension that is ready
        to be used in Tesseract training (.gt.txt).
        Tries to crop the image so that only the relevant text gets passed to Tesseract.
        Returns the name of the text file that contains the text.
        """
        #file_path = '/home/vimal/Documents/table_detect_samples/structured images/input/011364700/cells'
        #image_file = os.listdir(file_path)

        for f in image_file:
            #print(f)
            directory, filename = os.path.split(f)
            filename_sans_ext, ext = os.path.splitext(filename)
            image = cv2.imread(os.path.join(file_path,f), cv2.IMREAD_GRAYSCALE)
            cropped = crop_to_text(image)
            #ocr_data_dir = os.path.join(directory, "ocr_data")
            #os.makedirs(ocr_data_dir, exist_ok=True)
            #out_imagepath = os.path.join(ocr_data_dir, filename)
            out_path = os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input',
                                    os.path.splitext(file_name)[0],'text')
            out_txtpath = os.path.join(out_path, "{}.txt".format(filename_sans_ext))
            #cv2.imwrite(out_imagepath, cropped)
            if not tess_args:
                d = os.path.dirname(sys.modules["table_ocr"].__file__)
                tessdata_dir = os.path.join(d, "tessdata")
                tess_args = ["--psm", "11", "-l", "table-ocr", "--tessdata-dir", tessdata_dir]
            txt = ocr_image(cropped, " ".join(tess_args))
            with open(out_txtpath, "w") as txt_file:
                txt_file.write(txt)
            #return out_txtpath
    def crop_to_text(image):
        MAX_COLOR_VAL = 255
        BLOCK_SIZE = 15
        SUBTRACT_FROM_MEAN = -2

        img_bin = cv2.adaptiveThreshold(
            ~image,
            MAX_COLOR_VAL,
            cv2.ADAPTIVE_THRESH_MEAN_C,
            cv2.THRESH_BINARY,
            BLOCK_SIZE,
            SUBTRACT_FROM_MEAN,
        )

        img_h, img_w = image.shape
        horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (int(img_w * 0.5), 1))
        vertical_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, int(img_h * 0.7)))
        horizontal_lines = cv2.morphologyEx(img_bin, cv2.MORPH_OPEN, horizontal_kernel)
        vertical_lines = cv2.morphologyEx(img_bin, cv2.MORPH_OPEN, vertical_kernel)
        both = horizontal_lines + vertical_lines
        cleaned = img_bin - both

        # Get rid of little noise.
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
        opened = cv2.morphologyEx(cleaned, cv2.MORPH_OPEN, kernel)
        opened = cv2.dilate(opened, kernel)

        contours, hierarchy = cv2.findContours(opened, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
        bounding_rects = [cv2.boundingRect(c) for c in contours]
        NUM_PX_COMMA = 6
        MIN_CHAR_AREA = 5 * 9
        char_sized_bounding_rects = [(x, y, w, h) for x, y, w, h in bounding_rects if w * h > MIN_CHAR_AREA]
        if char_sized_bounding_rects:
            minx, miny, maxx, maxy = math.inf, math.inf, 0, 0
            for x, y, w, h in char_sized_bounding_rects:
                minx = min(minx, x)
                miny = min(miny, y)
                maxx = max(maxx, x + w)
                maxy = max(maxy, y + h)
            x, y, w, h = minx, miny, maxx - minx, maxy - miny
            cropped = image[y:min(img_h, y+h+NUM_PX_COMMA), x:min(img_w, x+w)]
        else:
            # If we morphed out all of the text, assume an empty image.
            cropped = MAX_COLOR_VAL * np.ones(shape=(20, 100), dtype=np.uint8)
        bordered = cv2.copyMakeBorder(cropped, 5, 5, 5, 5, cv2.BORDER_CONSTANT, None, 255)
        return bordered
    def ocr_image(image, config):
        return pytesseract.image_to_string(
            image,
            lang='eng', config='psm--11'
        )

    file_path = os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input',
                                    os.path.splitext(file_name)[0],'cells')
    image_file = os.listdir(file_path)
    tess_args = os.listdir(file_path)

    main(image_file, tess_args)

    def text_files_to_csv(files):
        """Files must be sorted lexicographically
        Filenames must be <row>-<colum>.txt.
        000-000.txt
        000-001.txt
        001-000.txt
        etc...
        """
        rows = []
        for f in files:
            directory, filename = os.path.split(f)
            with open(os.path.join(file_path,f)) as of:
                txt = of.read().strip()
            row, column = map(int, filename.split(".")[0].split("-"))
            if row == len(rows):
                rows.append([])
            rows[row].append(txt)

        csv_file = io.StringIO()
        writer = csv.writer(csv_file)
        writer.writerows(rows)
        return csv_file.getvalue()

    def main(files):
        return text_files_to_csv(files)


    file_path = os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input',
                                    os.path.splitext(file_name)[0],'text')
    file = os.listdir(file_path)
    files = sorted(file)

    a = main(files)
    #print(a)

    c = os.path.splitext(file_name)[0]+'_text.txt'
    t = os.path.splitext(file_name)[0]+'_f-text.txt'

    #cs = os.path.splitext(file_name)[0]+'_xl.csv'

    text_file = open(os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input',
                                    os.path.splitext(file_name)[0],'txt',c), "wt")
    n = text_file.write(a)
    text_file.close()

    df = pd.read_csv(os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input',
                                    os.path.splitext(file_name)[0],'txt',c),
            header=None,delimiter=',',names=list(range(5)))
    df[1] = df[1].astype(str)
    #print(df)

    df1 = df.dropna(axis = 0,how = 'all')
    #print(df1)

    word = 'TOTAL WAGES PAID'



    if df1[0].dtypes != 'object':
        print(os.path.splitext(file_name)[0]+'.tif','TOTAL WAGES PAID',0)

    else:

        for i in df1[0]:
            if fuzz.ratio(word,str(i)) > 80:
                #print(i)
                dff = df1[df1[0] == i]
                dff = dff.dropna(axis = 1,how = 'all')
                output = dff[1]
                output = output.str.replace(r'[^\w\s]+', '')
                output = output.str.replace(" ","")
                #print(output)

                if len(output) == 0:
                    print(os.path.splitext(file_name)[0]+'.tif','TOTAL WAGES PAID',0)
                else:
                    with open(os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input',
                                                os.path.splitext(file_name)[0],'final_text',t), 'a') as f:
                        f.write(output.to_string(header = False, index = False))

                    text_file = open(os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/table-ocr_input',
                                                os.path.splitext(file_name)[0],'final_text',t), 'r+')

                    print(os.path.splitext(file_name)[0]+'.tif','TOTAL WAGES PAID',text_file.read())
        
                
                
    
            
        

In [2]:
file_name = '0423173000.jpeg'
twp(file_name)

/home/vimalkumar.s/Documents/python_new/cde_venv/lib/python3.8/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


0423173000.tif TOTAL WAGES PAID 4963118


In [3]:
file_name = '0319768000.jpeg'
twp(file_name)

0319768000.tif TOTAL WAGES PAID 385196
